In [1]:
from PyPDF2 import PdfReader
import re
import string
from os import scandir, getcwd
from datetime import datetime
import spacy
import pandas as pd
import pickle
import dask.dataframe as dd
from py2neo import Graph
import textstat
textstat.set_lang('es')

In [2]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "congreso"))

In [13]:
graph.run(
        'MATCH (n:Hablante) \
         MATCH (p:Palabra) \
         WHERE n.nombre =  $nombre \
         AND p.lemma = $lemma \
         CREATE (n)-[r:DICE {fecha: $fecha, veces: $veces, grupo_parlamentario: $grupo}]->(p) \
         RETURN COUNT(r)',
        nombre='Bel Accensi, Ferran',
        lemma='práctica',
        fecha='2023-02-01',
        veces=1,
        grupo='aaaa'
    ).stats()

{'relationships_created': 1, 'properties_set': 3}

In [14]:
'     '.strip()

''

In [257]:
def ls(ruta=getcwd() + '\\documentos'):
    return [arch.path for arch in scandir(ruta) if arch.is_file()]

In [258]:
def extrae_paginas(reader):
    paginas = []
    #extraemos las páginas y eliminamos las cabeceras, y conservamos la fecha del debate
    for i in range(len(reader.pages)):
        if i == 0:
            paginas.extend(reader.pages[i].extract_text().split('\n')[9:])
        else:
            if i == 1:
                fecha_proceso = reader.pages[i].extract_text().split('\n')[2]
            paginas.extend(reader.pages[i].extract_text().split('\n')[3:])
    return paginas, fecha_proceso

In [259]:
def procesa_paginas(paginas, documento):
    paginas_procesadas = []
    linea_aux = ''
    for linea in paginas:
        if linea.startswith('cve: DSCD'):
            continue
        else:
            linea_limpia = linea.replace('\u2002', ' ').replace('.', '').replace(chr(8230), '')
            linea_aux = linea_aux + linea_limpia
            if (len(linea_limpia) == 0) or linea_limpia[-1] != ' ':
                #añado una excepcion, se ha detectado que en un documento falta el símbolo ':' después de un hablante
                if documento == r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-94.PDF' and \
                linea_aux.startswith('El señor PRESIDENTE DEL GOBIERNO  (Sánchez Pérez-Castejón) La diferencia, señor Abascal, señorías de VOX,'):
                    linea_aux = linea_aux.replace('(Sánchez Pérez-Castejón) La diferencia,', '(Sánchez Pérez-Castejón): La diferencia,')
                paginas_procesadas.append(linea_aux)
                linea_aux = ''
    return paginas_procesadas

In [260]:
def limpia_paginas(paginas_procesadas):
    paginas_limpias = []
    for linea in paginas_procesadas:
        if linea == 'Página':
            continue
        else:
            paginas_limpias.append(linea)
    return paginas_limpias

In [261]:
def elimina_tildes(s):
    replacements = (
        ('á', 'a'),
        ('é', 'e'),
        ('í', 'i'),
        ('ó', 'o'),
        ('ú', 'u'),
        ('à', 'a'),
        ('è', 'e'),
        ('ì', 'i'),
        ('ò', 'o'),
        ('ù', 'u'),
        ('ü', 'u'),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [262]:
def transforma_hablante(linea):
    return elimina_tildes(linea.strip()\
                          .replace(' ', '')\
                          .replace('-', '')\
                          .replace(r'\x', '')\
                          .replace(',', '')\
                         ).lower()

In [263]:
def extrae_texto(paginas_limpias, documento):
    correccion_inicio = {
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-109.PDF':
        'Se abre la sesión a las nueve de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-12.PDF':
        'Se abre la sesión a las tres de la tarde',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-120.PDF':
        'Se abre la sesión a las tres de la tarde',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-143.PDF':
        'Se reanuda la sesión a las nueve de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-154.PDF':
        'Se abre la sesión a las tres de la tarde',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-156.PDF':
        'Se reanuda la sesión a la nueve de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-167.PDF':
        'Se abre la sesión a las tres de la tarde',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-18.PDF':
        'Se abre la sesión a las diez de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-192.PDF':
        'Se reanuda la sesión a las nueve de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-2.PDF':
        'Se abre la sesión a las nueve de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-202.PDF':
        'Se reanuda la sesión a las diez de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-204.PDF':
        'Se abre la sesión a las nueve de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-205.PDF':
        'Se abre la sesión a las tres de la tarde',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-215.PDF':
        'Se abre la sesión a las nueve de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-217.PDF':
        'Se abre la sesión a las nueve de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-231.PDF':
        'Se abre la sesión a las tres de la tarde',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-27.PDF':
        'Se abre la sesión a las diez de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-35.PDF':
        'Se levanta la sesión a las cuatro y quince minutos de la tarde Se reanuda la sesión a las nueve de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-4.PDF':
        'Se reanuda la sesión a las doce del mediodía',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-41.PDF':
        'Se abre la sesión a las tres de la tarde',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-44.PDF':
        'Se abre la sesión a las tres de la tarde EXCLUSIÓN DEL ORDEN DEL DÍA:',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-45.PDF':
        'Se reanuda la sesión a las nueve de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-51.PDF':
        'Se abre la sesión a las tres de la tarde',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-59.PDF':
        'Se reanuda la sesión a la nueve de la mañana',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-75.PDF':
        'Se abre la sesión a las tres de la tarde',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-87.PDF':
        'Se abre la sesión a las tres de la tarde',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-9.PDF':
        'Se abre la sesión a las tres de la tarde',
}

    sw_sumario_encontrado = False
    sw_linea_inicio = False
    sw_inicio_encontrado = False
    linea_inicio = ''
    texto = []
    #Buscamos el inicio que está en la siguiente línea despues de SUMARIO
    for linea in paginas_limpias:
        if sw_sumario_encontrado == False:
            if linea == 'SUMARIO':
                sw_sumario_encontrado = True
        else:
            if sw_linea_inicio == False:
                try:
                    linea_inicio= correccion_inicio[documento]
                except:
                    linea_inicio = linea
                    
                sw_linea_inicio = True
            else:
                if sw_inicio_encontrado == False:
                    if linea == linea_inicio:
                        sw_inicio_encontrado = True
                else:
                    #eliminamos las que contienen texto en mayusculas
                    if not linea[:5].isupper():
                        texto.append(linea)
    return texto

In [264]:
def agrupa_dialogos(texto):
    #limpieza de texto
    #eliminamos todo lo que esté entre paréntesis
    texto_sin_parentesis= []
    #A veces es necesario forzar algunas líneas 
    excepciones_texto = ['La señora Cayetana Álvarez de Toledo se presentó p',
                         'La señora Arrimadas creo que ha hecho una interven',
                         'El señor Garzón seguirá siendo ministro de Consumo',
                         'El señor Rufián ha hecho una serie de referencias ',
                         'La señora ministra dijo la semana pasada en este h',
                         'La señora portavoz de Ciudadanos ha ido a descaro:',
                         'El señor Cabezón apuntaba antes que el señor minis',
                         'El señor Rego dice: necesitamos una nueva regulaci',
                         'El señor Abascal ha venido aquí a hablar de su lib',
                         'El señor Arnaldo tiene un montón de méritos que ha',
                         'La señora ministra, en el debate de convalidación ',
                         'El señor Grande-Marlaska, en una época en la que B',
                         'El señor Errejón vuelve a hablar de las medidas de',
                         'El señor Ortega Smith me decía cuando bajaba — al ',
                         'La señora Gamarra ha hecho una intervención y la v',
                         'El señor Rufián y la señora Nogueras me permitirán',
                         'El señor Echenique propagó una noticia falsa dicie',
                         'El señor Esteban me critica, con razón, la técnica',
                         'El señor Abascal, en un alarde más de testosterona',
                         'El señor  Bel me dice:  le ha faltado empatía Lo h',
                         'El señor Abascal tiende a la abstracción siempre q',
                         'La señora diputada Dantas del Grupo Republicano, n',
                         'La señora Oramas ha manifestado su voto en contra ',
                         'El señor Iglesias ahora se ríe, pero recuerde cómo',
                         'La señora Gamarra poco menos que viene a decir que',
                         'La señora Gamarra también dice que no estamos haci',
                         'El señor Rojas también ha dicho una cosa que me ha',
                         'La señora Noguera (sic) ha hecho referencia tambié',
                         'La señora Díaz considera cada nueva EPA una muestr',
                         'La señora Castillo dice que es una chapuza legisla',
                         'El señor Casado nos tiene acostumbrados a algo que',
                         'La señora Toscano en su intervención decía lo que ',
                         'El señor Sayas siempre me habla de ETA y luego me ',
                         'El señor Baldoví ya viene con el traje puesto de c',
                         'El señor de VOX ha hecho referencia a las cuatro v',
                         'El señor Sayas dice que jamás hubiera aceptado los',
                         'El señor portavoz de Podemos no ha mencionado a ni',
                         'El señor Echenique creo que ha hecho un planteamie',
                         'El señor Abascal siente que esta Cámara elegida co',
                         'El señor Sánchez lleva casi cincuenta reales decre',
                         'El señor Pérez Bueno cuando vino en octubre a esta',
                         'El señor Paniagua ―otro argumento que también le c',
                         'El señor Iceta ha dicho que ha nacido para pactar,',
                         'La señora Aizpurua viene aquí y nos cuenta una par',
                         'El señor Sánchez ―que siento mucho que no esté aqu',
                         'El señor ministro tiene una agenda y estos días se',
                         'El señor Sánchez ha dicho una cosa muy interesante',
                         'El señor Bel me preguntaba cuál es la alternativa ',
                         'La señora Gamarra se reúne con una asociación o pl',
                         'El señor Zapico se refería a la necesaria certeza,',
                        ]

    # Agrupamos discursos, un discurso empieza con el texto 'La señora' o 'El señor', asumimos que lo que no comience 
    # por esto, pertenecerá al anterior hablante detectado
    dialogos_agrupados = []
    dialogo = ''
    for linea in texto:
        if (':' in linea and (linea.startswith('La señora') or linea.startswith('El señor'))) and \
        (linea[:50] not in excepciones_texto):
            dialogos_agrupados.append(" ".join(dialogo.split()))
            dialogo = ''
        dialogo = dialogo + ' ' + linea
    return dialogos_agrupados

In [265]:
def remove_punctuation ( text ):
    simbolos = '¿?¡!-0123456789‘’―—─‒‹›<>°´ºª'
    output = re.sub(r'\(.*?\)', '', text)
    output = re.sub('[%s]' % re.escape(string.punctuation), '', output)
    output = re.sub('[%s]' % re.escape(simbolos), '', output)
    return output

In [266]:
def devuelve_fecha(cadena):
    meses = {
        'enero': '01',
        'febrero': '02',
        'marzo': '03',
        'abril': '04',
        'mayo': '05',
        'junio': '06',
        'julio': '07',
        'agosto': '08',
        'septiembre': '09',
        'octubre': '10',
        'noviembre': '11',
        'diciembre': '12'
    }
    temp = cadena.split(' ')

    fecha = temp[7] + '-' + meses[temp[5].lower()] + '-' + temp[3].rjust(2, '0')
    valida = datetime.strptime(fecha, '%Y-%m-%d')
    return fecha
        

In [267]:
def elimina_puntuacion(dialogos_agrupados, fecha_proceso):
    dialogos = []
    for linea in dialogos_agrupados:
        ix = linea.find(':')
        excluidos = [
            'elseñorvicepresidentegracias(rodriguezgomezdecelis)',
            'laseñoravicepresidentasegunda(pastorjulian)',
            'elseñorvicepresidente(rodriguezdecelis)',
            'elseñorvicepresidente',
            'laseñorasecretaria(hernanzcosta)',
            'elseñorpresidente(rodriguezgomezdecelis)',
            'elseñorrodriguezgomezdecelis',
            'elseñorvicepresidente(rodriguezgomezdecelis',
            'laseñorapresidente',
            'elseñorsecretario(sanchezserna)',
            'laseñoravicepresidente(pastorjulian)',
            'laseñoravicepresidenta(elizoserrano)',
            'elseñorvicepresidente(gomezdecelis)',
            'laseñorapresienta',
            'laseñorapresidenta',
            'elseñorvicepresidente(gillazaro)',
            'laseñoravicepresidenta(pastorjulian)',
            'elseñorvicepresidente(gomezrodriguezdecelis)',
            'laseñoravicepresidenta(julianpastor)',
            'elseñorpresidente',
            'elseñorsecretario(pisarelloprados)',
            'elseñorpresidenta',
            'elseñorsecretario(pisareloprados)',
            'laseñorapresidena',
            'elseñorvicepresidentegillazaro',
            'elseñorvicepresidente(rodriguezgomezdecelis)',
            'elseñorsecretario(suarezillana)',
            'elseñorsuarezillana',
            'elseñorvicepresidente(rodriguezgomezcelis)',
            'laseñorasecretaria(hernazcosta)',
            'laseñoravicepresidenta',
            'elseñorvicpresidente(rodriguezgomezdecelis)',
            'elseñorvicepresidenteprimero(rodriguezgomezdecelis)',
            'laseñoraviceresidenta(elizoserrano)'
        ]

        if ix != -1:
            if transforma_hablante(linea[:ix]) not in excluidos:
                dialogos.append({'hablante': linea[:ix].strip(), 
                                 'hablante_transformado': transforma_hablante(linea[:ix]),
#                                 'discurso': " ".join(remove_punctuation(linea[ix +1:]).split()),
                                 'discurso': linea[ix +1:],
                                 'fecha_proceso': devuelve_fecha(fecha_proceso)})
    return dialogos

In [268]:
dict_hablantes = {
    'laseñorauriartebengoechea': {'nombre': 'Uriarte Bengoechea, Edurne',
                                  'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorpisarelloprados': {'nombre': 'Pisarello Prados, Gerardo',
                               'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñormestrebarea': {'nombre': 'Mestre Barea, Manuel',
                           'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoratelecheailozano': {'nombre': 'Telechea i Lozano, Carolina',
                                'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'laseñoratelechealozano': {'nombre': 'Telechea i Lozano, Carolina',
                                'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'laseñoraministradetrabajoyeconomiasocial': {'nombre': 'Díaz Pérez, Yolanda',
                                                 'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoravicepresidentaterceradelgobiernoyministradetrabajoyeconomiasocial(diazperez)': {
        'nombre': 'Díaz Pérez, Yolanda',
        'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoravicepresidentatercerayministradetrabajoyeconomiasocial(diazperez)': {
        'nombre': 'Díaz Pérez, Yolanda',
        'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoravicepresidentasegundadelgobiernoyministradetrabajoyeconomiasocial(diazperez)': {
        'nombre': 'Díaz Pérez, Yolanda',
        'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoravicepresidentasegundayministradetrabajoyeconomiasocial(diazperez)': {
        'nombre': 'Díaz Pérez, Yolanda',
        'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoraministradetrabajoyeconomiasocial(diazperez)': {'nombre': 'Díaz Pérez, Yolanda',
                                                            'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoravicepresidentasegundayministradetrabajoyecoconomiasocial(diazperez)': {
        'nombre': 'Díaz Pérez, Yolanda',
        'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoraministrodetrabajoyeconomiasocial(diazperez)': {
        'nombre': 'Díaz Pérez, Yolanda',
        'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoravicepresidentasegundayministradetrabajoyeconomiasocial': {'nombre': 'Díaz Pérez, Yolanda',
                                                                       'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorloritelorite': {'nombre': 'Lorite Lorite, Andrés',
                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorministrodelinterior(grandemarlaskagomez)': {'nombre': 'Grande-Marlaska Gómez, Fernando',
                                                        'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorministrodelinterior(grandemarlaska)': {'nombre': 'Grande-Marlaska Gómez, Fernando',
                                                   'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorministrodeinterior(grandemarlaskagomez)': {'nombre': 'Grande-Marlaska Gómez, Fernando',
                                                       'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorministeriodelinterior(grandemarlaskagomez)': {'nombre': 'Grande-Marlaska Gómez, Fernando',
                                                       'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñormatutegarciadejalon': {'nombre': 'Matute García de Jalón, Oskar',
                                   'grupo parlamentario': 'Grupo Parlamentario Euskal Herria Bildu'},
    'elseñormatutuegarciadejalon': {'nombre': 'Matute García de Jalón, Oskar',
                                    'grupo parlamentario': 'Grupo Parlamentario Euskal Herria Bildu'},
    'elseñormatutegarciadeljalon': {'nombre': 'Matute García de Jalón, Oskar',
                                    'grupo parlamentario': 'Grupo Parlamentario Euskal Herria Bildu'},
    'laseñoraminguezgarcia': {'nombre': 'Mínguez García, Montse',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorgarrigavazdeconcicao': {'nombre': 'Garriga Vaz de Concicao, Ignacio',
                                    'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoracarvalhodantas': {'nombre': 'Carvalho Dantas, María',
                               'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'laseñoracalvalhodantas': {'nombre': 'Carvalho Dantas, María',
                               'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'laseñoravehicantenys': {'nombre': 'Vehí Cantenys, Mireia',
                             'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'laseñoravehicanteys': {'nombre': 'Vehí Cantenys, Mireia',
                            'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'laseñoravehiicantenys': {'nombre': 'Vehí Cantenys, Mireia',
                              'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'elseñorestebanbravo': {'nombre': 'Esteban Bravo, Aitor',
                            'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñorestabanbravo': {'nombre': 'Esteban Bravo, Aitor',
                            'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñorpresidentedelgobierno(sanchezperezcastejon)': {'nombre': 'Sánchez Pérez-Castejón, Pedro',
                                                           'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsanchezperezcastejon(candidatoalapresidenciadelgobierno)': {
        'nombre': 'Sánchez Pérez-Castejón, Pedro',
        'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorpresidentedegobierno(sanchezperezcastejon)': {'nombre': 'Sánchez Pérez-Castejón, Pedro',
                                                          'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorpresidentedelgobierno': {'nombre': 'Sánchez Pérez-Castejón, Pedro',
                                     'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsantiagoromero': {'nombre': 'Santiago Romero, Enrique Fernando',
                              'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorlegardauriarte': {'nombre': 'Legarda Uriarte, Mikel',
                              'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñorlegardeuriarte': {'nombre': 'Legarda Uriarte, Mikel',
                              'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñorlegardaurigarte': {'nombre': 'Legarda Uriarte, Mikel',
                               'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñorbelaccensi': {'nombre': 'Bel Accensi, Ferran',
                          'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'elseñorbelacccensi': {'nombre': 'Bel Accensi, Ferran',
                           'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'elseñorguitartegimeno': {'nombre': 'Guitarte Gimeno, Tomás',
                              'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'elseñorfernandezhernandez': {'nombre': 'Fernández Hernández, Pedro',
                                  'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoraministradepoliticaterritorialyfuncionpublica(dariassansebastian)': {
        'nombre': 'Darias San Sebastián, Carolina',
        'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradesanidad(dariassansebastian)': {'nombre': 'Darias San Sebastián, Carolina',
                                                      'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñoralonsocuevillasisayrol': {'nombre': 'Alonso-Cuevillas i Sayrol, Jaume',
                                      'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'laseñoracalvopoyato': {'nombre': 'Calvo Poyato, Carmen',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentaprimeradelgobiernoyministradelapresidenciarelacionesconlascortesymemoriademocratica(calvopoyato)': {
        'nombre': 'Calvo Poyato, Carmen', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentadelgobiernoyministradelapresidenciarelacionesconlascortesymemoriademocratica(calvopoyato)': {
        'nombre': 'Calvo Poyato, Carmen', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentaprimeradelgobiernoyministradelapresidenciarelacionesconlascorteseigualdad(calvopoyato)': {
        'nombre': 'Calvo Poyato, Carmen', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentaprimeradelgobiernoyministradelapresdienciarelacionesconlascortesymemoriademocratica(calvopoyato)': {
        'nombre': 'Calvo Poyato, Carmen', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentaprimerayministradelapresidenciarelacionesconlascortesymemoriademocratica(calvopoyato)': {
        'nombre': 'Calvo Poyato, Carmen', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentaprimeradelgobiernoministradelapresidenciarelacionesconlascortesymemoriademocratica(calvopoyato)': {
        'nombre': 'Calvo Poyato, Carmen', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentaprimeradelgobiernoyministradelapresidenciarelacionesconlascortesymemoriademocrática(calvopoyato)': {
        'nombre': 'Calvo Poyato, Carmen', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentaprimeraministradelapresidenciarelacionesconlascortesymemoriademocratica(calvopoyato)': {
        'nombre': 'Calvo Poyato, Carmen', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradedefensa': {'nombre': 'Robles Fernández, Margarita',
                                  'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradedefensa(roblesfernandez)': {'nombre': 'Robles Fernández, Margarita',
                                                   'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradehacienda(monterocuadrado)': {'nombre': 'Montero Cuadrado, María Jesús',
                                                    'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradehacienda(montero)': {'nombre': 'Montero Cuadrado, María Jesús',
                                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradehaciendayfuncionpublica(monterocuadrado)': {
        'nombre': 'Montero Cuadrado, María Jesús',
        'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradehaciendayfuncionpública(monterocuadrado)': {
        'nombre': 'Montero Cuadrado, María Jesús',
        'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministrahaciendayfuncionpublica(monterocuadrado)': {'nombre': 'Montero Cuadrado, María Jesús',
                                                                 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradehaciendayportavozdelgobierno(monterocuadrado)': {'nombre': 'Montero Cuadrado, María Jesús',
                                                                 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorgarciaadanero': {'nombre': 'García Adanero, Carlos',
                             'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'laseñoraalvarezdetoledoperaltaramos': {'nombre': 'Álvarez de Toledo Peralta-Ramos, Cayetana',
                                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoraalvarezdetoledoperalta\xadramos': {'nombre': 'Álvarez de Toledo Peralta-Ramos, Cayetana',
                                                'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorbalfrances': {'nombre': 'Bal Francés, Edmundo',
                          'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'elseñorbalfrancés': {'nombre': 'Bal Francés, Edmundo',
                           'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'elseñorgarciadiez': {'nombre': 'García Díez, Joaquín María',
                          'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñoriñarritugarcia': {'nombre': 'Iñarritu García, Jon',
                              'grupo parlamentario': 'Grupo Parlamentario Euskal Herria Bildu'},
    'elseñormargallsastre': {'nombre': 'Margall Sastre, Joan',
                             'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'elseñorcastellonrubio': {'nombre': 'Castellón Rubio, Miguel Ángel',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoraborrascantanyer': {'nombre': 'Borràs Castanyer, Laura',
                                'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'laseñoraborrascastanyer': {'nombre': 'Borràs Castanyer, Laura',
                                'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'laseñoraborràscastanyer': {'nombre': 'Borràs Castanyer, Laura',
                                 'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'laseñoraruedaperello': {'nombre': 'Rueda Perelló, Patricia',
                             'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoravicepresidentacuartadelgobiernoyministraparalatransicionecologicayelretodemografico(riberarodriguez)': {
        'nombre': 'Ribera Rodríguez, Teresa', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentacuartadelgobiernoyministraparalatransicionecologicayelretodemográfico(riberarodriguez)': {
        'nombre': 'Ribera Rodríguez, Teresa', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentacuartayministraparalatransicionecologicayelretodemografico(riberarodriguez)': {
        'nombre': 'Ribera Rodríguez, Teresa', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentatercerayministraparalatransicionecologicayelretodemografico(riberarodriguez)': {
        'nombre': 'Ribera Rodríguez, Teresa', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentaterceradelgobiernoyministraparalatransicionecologicayelretodemografico(riberarodriguez)': {
        'nombre': 'Ribera Rodríguez, Teresa', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentatercerayministraparalatransicionecologicayretodemografico(riberarodriguez)': {
        'nombre': 'Ribera Rodríguez, Teresa', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentayministraparalatransicionecologicayelretodemografico(riberarodriguez)': {
        'nombre': 'Ribera Rodríguez, Teresa', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentatercerayministraparalatransicionecologicayelretodemografico(riberarodriguez)': {
        'nombre': 'Ribera Rodríguez, Teresa', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentatercerayministradelatransicionecologicayelretodemografico(riberarodriguez)': {
        'nombre': 'Ribera Rodríguez, Teresa', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentatercerayministraparalatransicionecologicayelretodemodragico(riberarodriguez)': {
        'nombre': 'Ribera Rodríguez, Teresa', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentacuartayministraparalatransicionecologicayelretodemografico(riverarodriguez)': {
        'nombre': 'Ribera Rodríguez, Teresa', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentaterceradelgobiernoyministradeasuntoseconomicosytransformaciondigital(calviñosantamaria)': {
        'nombre': 'Calviño Santamaría, Nadia', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentasegundadelgobiernoyministradeasuntoseconomicosytransformaciondigital(calviñosantamaria)': {
        'nombre': 'Calviño Santamaría, Nadia', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentaprimeradelgobiernoyministradeasuntoseconomicosytransformaciondigital(calviñosantamaria)': {
        'nombre': 'Calviño Santamaría, Nadia', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentasegundayministradeasuntoseconomicosytransformaciondigital(calviñosantamaria)': {
        'nombre': 'Calviño Santamaría, Nadia', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentaprimerayministradeasuntoseconomicosytransformaciondigital(calviñosantamaria)': {
        'nombre': 'Calviño Santamaría, Nadia', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentaprimerayministraasuntoseconomicosytransformaciondigital(calviñosantamaria)': {
        'nombre': 'Calviño Santamaría, Nadia', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresientaprimeradelgobiernoyministradeasuntoseconomicosytransformaciondigital(calviñosantamaria)': {
        'nombre': 'Calviño Santamaría, Nadia', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentaterceradelgobiernoyministradeasuntoseconomicosytransformaciondigital(calvinosantamaria)': {
        'nombre': 'Calviño Santamaría, Nadia', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresdientaprimerayministradeasuntoseconomicosytransformaciondigital(calviñosantamaria)': {
        'nombre': 'Calviño Santamaría, Nadia', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentatercerayministradeasuntoseconomicosytransformaciondigital(calviñosantamaria)': {
        'nombre': 'Calviño Santamaría, Nadia', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravicepresidentadelgobiernoyministradeasuntoseconomicosytransformaciondigital(calviñosantamaria)': {
        'nombre': 'Calviño Santamaría, Nadia', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorbotranpahissa': {'nombre': 'Botran Pahissa, Albert',
                             'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'elseñorbotránpahissa': {'nombre': 'Botran Pahissa, Albert',
                              'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'elseñorbotranipahissa': {'nombre': 'Botran Pahissa, Albert',
                              'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'elseñorbaldoviroda': {'nombre': 'Baldoví Roda, Joan',
                           'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'elseñorbaldoviiroda': {'nombre': 'Baldoví Roda, Joan',
                            'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'elseñorbaldobiroda': {'nombre': 'Baldoví Roda, Joan',
                           'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'laseñoragamarraruizclavijo': {'nombre': 'Gamarra Ruiz-Clavijo, Concepción',
                                   'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñorarayarodriguez': {'nombre': 'Raya Rodríguez, María Tamara',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorpirizmaya': {'nombre': 'Píriz Maya, Víctor Valentín',
                         'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorespejosaavedraconesa': {'nombre': 'Espejo-Saavedra Conesa, José María',
                                    'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'elseñorutrillacano': {'nombre': 'Utrilla Cano, Julio',
                           'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorelorzagonzalez': {'nombre': 'Elorza González, Odón',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorelorzagonzález': {'nombre': 'Elorza González, Odón',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradetransportes,movilidadyagendaurbana': {'nombre': 'Sánchez Jiménez, Raquel',
                                                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradetransportesmovililidadyagendaurbana(sanchezjimenez)': {
        'nombre': 'Sánchez Jiménez, Raquel', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradetransportesmovilidadyagendaurbana(sanchezjimenez)': {
        'nombre': 'Sánchez Jiménez, Raquel', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorcasadoblanco': {'nombre': 'Casado Blanco, Pablo',
                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñordequintoromero': {'nombre': 'De Quinto Romero, Marcos',
                              'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'elseñorquevedoiturbe': {'nombre': 'Quevedo Iturbe, Pedro',
                             'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'elseñorquevedoiburbe': {'nombre': 'Quevedo Iturbe, Pedro',
                             'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'laseñoraarrimadasgarcia': {'nombre': 'Arrimadas García, Inés',
                                'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'elseñormazonramos': {'nombre': 'Mazón Ramos, José María',
                          'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'elseñoragomezreinovarela': {'nombre': 'Gómez-Reino Varela, Antonio',
                                 'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorgomezreinovarela': {'nombre': 'Gómez-Reino Varela, Antonio',
                                'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorasensllodra': {'nombre': 'Asens Llodrà, Jaume',
                           'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorrufianromero': {'nombre': 'Rufián Romero, Gabriel',
                            'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'elseñorrufiánromero': {'nombre': 'Rufián Romero, Gabriel',
                             'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'elseñorrufianramos': {'nombre': 'Rufián Romero, Gabriel',
                           'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'laseñorasabanesnadal': {'nombre': 'Sabanés Nadal, Inés',
                             'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'laseñorasabanes': {'nombre': 'Sabanés Nadal, Inés',
                        'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'laseñoraoramasgonzalezmoro': {'nombre': 'Oramas González-Moro, Ana María',
                                   'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'laseñoraoramasgonzálezmoro': {'nombre': 'Oramas González-Moro, Ana María',
                                    'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'elseñorabascalconde': {'nombre': 'Abascal Conde, Santiago',
                            'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorcandidatoapresidentedelgobierno(abascalconde)': {'nombre': 'Abascal Conde, Santiago',
                                                             'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorcandidatoalapresidenciadelgobierno(abascalconde)': {'nombre': 'Abascal Conde, Santiago',
                                                                'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorregocandamil': {'nombre': 'Rego Candamil, Néstor',
                            'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'elseñorgarzonespinosa': {'nombre': 'Garzón Espinosa, Alberto',
                              'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorministrodeconsumo(garzonespinosa)': {'nombre': 'Garzón Espinosa, Alberto',
                                                 'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñoriglesiasturrion': {'nombre': 'Iglesias Turrión, Pablo',
                               'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorvicepresidentesegundodelgobiernoyministrodederechossocialesyagenda2030(iglesiasturrion)': {
        'nombre': 'Iglesias Turrión, Pablo',
        'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorvicepresidentesegundodelgobiernoyministrodederechosocialesyagenda2030(iglesiasturrion)': {
        'nombre': 'Iglesias Turrión, Pablo',
        'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorvicepresidentesegundoyministrodederechossocialesyagenda2030(iglesiasturrion)': {
        'nombre': 'Iglesias Turrión, Pablo',
        'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorvicepresidentedelgobiernoyministrodederechossocialesyagenda2030(iglesiasturrion)': {
        'nombre': 'Iglesias Turrión, Pablo',
        'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorsayaslopez': {'nombre': 'Sayas López, Sergio',
                          'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'elseñormartinezoblanca': {'nombre': 'Martínez Oblanca, Isidro Manuel',
                               'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'laseñoraaizpuruaarzallus': {'nombre': 'Aizpurua Arzallus, Mertxe',
                                 'grupo parlamentario': 'Grupo Parlamentario Euskal Herria Bildu'},
    'laseñoraazpiruaarzallus': {'nombre': 'Aizpurua Arzallus, Mertxe',
                                'grupo parlamentario': 'Grupo Parlamentario Euskal Herria Bildu'},
    'laseñoraaizpuruaarzalluz': {'nombre': 'Aizpurua Arzallus, Mertxe',
                                'grupo parlamentario': 'Grupo Parlamentario Euskal Herria Bildu'},
    'laseñoralastrafernandez': {'nombre': 'Lastra Fernández, Adriana',
                                'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoralastrafernández': {'nombre': 'Lastra Fernández, Adriana',
                                 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorespinosadelosmonterosdesimon': {'nombre': 'Espinosa de los Monteros de Simón, Iván',
                                            'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorespinosadelosmonteros': {'nombre': 'Espinosa de los Monteros de Simón, Iván',
                                     'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorerrejongalvan': {'nombre': 'Errejón Galván, Íñigo',
                             'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'elseñorerrejongalván': {'nombre': 'Errejón Galván, Íñigo',
                              'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'laseñorabassacoll': {'nombre': 'Bassa Coll, Montserrat',
                          'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'elseñorgutierrezsalinas': {'nombre': 'Gutiérrez Salinas, Indalecio',
                                'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorguijarrogarcia': {'nombre': 'Guijarro García, Txema',
                              'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorministrodeinclusionseguridadsocialymigraciones(escrivabelmonte)': {
        'nombre': 'Escrivá Belmonte, José Luis', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoragimenezgimenez': {'nombre': 'Giménez Giménez, Sara',
                               'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'laseñoragimemezgimenez': {'nombre': 'Giménez Giménez, Sara',
                               'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'elseñorruizdepinedoundiano': {'nombre': 'Ruiz de Pinedo Undiano, Iñaki',
                                   'grupo parlamentario': 'Grupo Parlamentario Euskal Herria Bildu'},
    'elseñorruizdepindeoundiano': {'nombre': 'Ruiz de Pinedo Undiano, Iñaki',
                                   'grupo parlamentario': 'Grupo Parlamentario Euskal Herria Bildu'},
    'laseñorapitacardenes': {'nombre': 'Pita Cárdenes, María del Carmen',
                             'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'laseñorapitacárdenes': {'nombre': 'Pita Cárdenes, María del Carmen',
                              'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'laseñoraolonachoclan': {'nombre': 'Olona Choclán, Macarena',
                             'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoraolonachoclán': {'nombre': 'Olona Choclán, Macarena',
                              'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorgarcessanagustin': {'nombre': 'Garcés Sanagustín, Mario',
                                'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorgarcéssanagustin': {'nombre': 'Garcés Sanagustín, Mario',
                                 'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorgonzalezcoellodeportugal': {'nombre': 'González Coello de Portugal, Víctor',
                                        'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorgonzalescoellodeportugal': {'nombre': 'González Coello de Portugal, Víctor',
                                        'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorrodriguezrodriguez': {'nombre': 'Rodríguez Rodríguez, Alberto',
                                  'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoradelunatobarra': {'nombre': 'De Luna Tobarra, Llanos',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorcasareshontañon': {'nombre': 'Casares Hontañón, Pedro',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñoranguitaperez': {'nombre': 'Anguita Pérez, Omar',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsimancassimancas': {'nombre': 'Simancas Simancas, Rafael',
                                'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoramedelperez': {'nombre': 'Medel Pérez, Rosa María',
                           'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorboadellaesteve': {'nombre': 'Boadella Esteve, Genís',
                              'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'elseñorsiciliaalferez': {'nombre': 'Sicilia Alférez, Felipe Jesús',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsiliciaalferez': {'nombre': 'Sicilia Alférez, Felipe Jesús',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorasagastizabalunzetabarrenetxea': {'nombre': 'Sagastizabal Unzetabarrenetxea, Idoia',
                                              'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'laseñorasagastizabalunzetabarrenextea': {'nombre': 'Sagastizabal Unzetabarrenetxea, Idoia',
                                              'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'laseñorasagastizabalunzetaberrenetxea': {'nombre': 'Sagastizabal Unzetabarrenetxea, Idoia',
                                              'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'laseñorasagastizabalunzetabbarrenetxea': {'nombre': 'Sagastizabal Unzetabarrenetxea, Idoia',
                                              'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñorfuentescurbelo': {'nombre': 'Fuentes Curbelo, Juan Bernardo',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraguerralopez': {'nombre': 'Guerra López, Sonia',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorcambroneropiqueras': {'nombre': 'Cambronero Piqueras, Pablo',
                                  'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'elseñorcambroneraspiqueras': {'nombre': 'Cambronero Piqueras, Pablo',
                                   'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'elseñorbarandiaranbenito': {'nombre': 'Barandiaran Benito, Íñigo',
                                 'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñorbariandaranbenito': {'nombre': 'Barandiaran Benito, Íñigo',
                                 'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñorbaradiaranbenito': {'nombre': 'Barandiaran Benito, Íñigo',
                                'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñorbarandarianbenito': {'nombre': 'Barandiaran Benito, Íñigo',
                                 'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'laseñorapereaiconillas': {'nombre': 'Perea i Conillas, María Mercè',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsalvadoriduch': {'nombre': 'Salvador i Duch, Jordi',
                             'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'elseñormenaarca': {'nombre': 'Mena Arca, Joan',
                        'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorbermudezdecastrofernandez': {'nombre': 'Bermúdez de Castro Fernández, José Antonio',
                                         'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorbermudezdecastro': {'nombre': 'Bermúdez de Castro Fernández, José Antonio',
                                'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorfigaredoalvarezsala': {'nombre': 'Figaredo Álvarez-Sala, José María',
                                   'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorfigaredoalvarezsalas': {'nombre': 'Figaredo Álvarez-Sala, José María',
                                   'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoramuñozvidal': {'nombre': 'Muñoz Vidal, María',
                           'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'laseñoralealfernandez': {'nombre': 'Leal Fernández, Isaura',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorechanizsalgado': {'nombre': 'Echániz Salgado, José Ignacio',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorechánizsalgado': {'nombre': 'Echániz Salgado, José Ignacio',
                               'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorlosadafernandez': {'nombre': 'Losada Fernández, José',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoragarridogutierrez': {'nombre': 'Garrido Gutiérrez, Pilar',
                                 'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoragarridogutiérrez': {'nombre': 'Garrido Gutiérrez, Pilar',
                                  'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñorasaavedramuñoz': {'nombre': 'Saavedra Muñoz, Marisa',
                              'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñoragirretxeaurresti': {'nombre': 'Agirretxea Urresti, Joseba Andoni',
                                 'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñoragirrexteaurresti': {'nombre': 'Agirretxea Urresti, Joseba Andoni',
                                 'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñoragirretexeaurresti': {'nombre': 'Agirretxea Urresti, Joseba Andoni',
                                  'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñoraguirretxeaurresti': {'nombre': 'Agirretxea Urresti, Joseba Andoni',
                                  'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñoragirretxeauresti': {'nombre': 'Agirretxea Urresti, Joseba Andoni',
                                  'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'elseñormiquelivalenti': {'nombre': 'Miquel i Valentí, Sergi',
                              'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'elseñormiquelivanlenti': {'nombre': 'Miquel i Valentí, Sergi',
                               'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'elseñormiquelivalienti': {'nombre': 'Miquel i Valentí, Sergi',
                               'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'laseñorapujolifarre': {'nombre': 'Pujol i Farré, Norma',
                            'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'elseñorsanchezserna': {'nombre': 'Sánchez Serna, Javier',
                            'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorsánchezserna': {'nombre': 'Sánchez Serna, Javier',
                             'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoramartinezgranados': {'nombre': 'Martínez Granados, María Carmen',
                                 'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'laseñoramaestromoliner': {'nombre': 'Maestro Moliner, Roser',
                               'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorministrodetransportesmovilidadyagendaurbana(abalosmeco)': {'nombre': 'Ábalos Meco, José Luis',
                                                                        'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorlopezmaraver': {'nombre': 'López Maraver, Ángel',
                            'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorministrodeagriculturapescayalimentacion(planaspuchades)': {'nombre': 'Planas Puchades, Luis',
                                                                        'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorministrodesanidad(illaroca)': {'nombre': 'Illa Roca, Salvador',
                                           'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorahoyojulia': {'nombre': 'Hoyo Juliá, Belén',
                          'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorgarciaegea': {'nombre': 'García Egea, Teodoro',
                          'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñoreritjaciuro': {'nombre': 'Eritja Ciuró, Francesc Xavier',
                           'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'laseñoraespañareina': {'nombre': 'España Reina, Carolina',
                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoramartinezferro': {'nombre': 'Martínez Ferro, María Valentina',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoraromerovilches': {'nombre': 'Romero Vilches, María de los Reyes',
                              'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoravazquezblanco': {'nombre': 'Vázquez Blanco, Ana Belén',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñorademeermendez': {'nombre': 'De Meer Méndez, Rocío',
                             'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoragranollerscunillera': {'nombre': 'Granollers Cunillera, Inés',
                                    'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'laseñoragorospeelezcano': {'nombre': 'Gorospe Elezcano, Josune',
                                'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'laseñoragorospelezcano': {'nombre': 'Gorospe Elezcano, Josune',
                               'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'laseñoraelezcanogorospe': {'nombre': 'Gorospe Elezcano, Josune',
                               'grupo parlamentario': 'Grupo Parlamentario Vasco (EAJ-PNV)'},
    'laseñorarosiqueisaltor': {'nombre': 'Rosique i Saltor, Marta',
                               'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'laseñoranavarrolacoba': {'nombre': 'Navarro Lacoba, Carmen',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñormayoralperales': {'nombre': 'Mayoral Perales, Rafael',
                              'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñorapozuetafernandez': {'nombre': 'Pozueta Fernández, Isabel',
                                 'grupo parlamentario': 'Grupo Parlamentario Euskal Herria Bildu'},
    'laseñorapozuelafernandez': {'nombre': 'Pozueta Fernández, Isabel',
                                 'grupo parlamentario': 'Grupo Parlamentario Euskal Herria Bildu'},
    'laseñorapozuetafernández': {'nombre': 'Pozueta Fernández, Isabel',
                                 'grupo parlamentario': 'Grupo Parlamentario Euskal Herria Bildu'},
    'laseñoramarcosortega': {'nombre': 'Marcos Ortega, Milagros',
                             'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoramoroalmaraz': {'nombre': 'Moro Almaraz, María Jesús',
                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorlopezalvarez': {'nombre': 'López Álvarez, Patxi',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoracañadellsalvia': {'nombre': 'Cañadell Salvia, Concep',
                               'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'elseñordiazgomez': {'nombre': 'Díaz Gómez, Guillermo',
                         'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'elseñorjosevelez': {'nombre': 'José Vélez, Rafael',
                         'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoralopezdominguez': {'nombre': 'López Domínguez, Laura',
                               'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñorapedrajasainz': {'nombre': 'Pedraja Sáinz, Raquel',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorapedrazasainz': {'nombre': 'Pedraja Sáinz, Raquel',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravallugerabalaña': {'nombre': 'Vallugera Balañà, Pilar',
                                'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'laseñoravalluguerabalaña': {'nombre': 'Vallugera Balañà, Pilar',
                                 'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'laseñorafrancocarmona': {'nombre': 'Franco Carmona, Isabel',
                              'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorchamorrodelmo': {'nombre': 'Chamorro Delmo, Ricardo',
                             'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñordelvalledeiscar': {'nombre': 'Del Valle de Iscar, Julio',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsarriamorell': {'nombre': 'Sarrià Morell, Vicent Manuel',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoragonzalezvazquez': {'nombre': 'González Vázquez, Marta',
                                'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñorafernandezcastañon': {'nombre': 'Fernández Castañón, Sofía',
                                  'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñorafernandezcatañon': {'nombre': 'Fernández Castañón, Sofía',
                                 'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñorafernándezcastañon': {'nombre': 'Fernández Castañón, Sofía',
                                   'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorgomezhernandez': {'nombre': 'Gómez Hernández, Héctor',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorgomezhernández': {'nombre': 'Gómez Hernández, Héctor',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorrojasgarcia': {'nombre': 'Rojas García, Carlos',
                           'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorfagundezcampo': {'nombre': 'Fagúndez Campo, Antidio',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorfagundezcampos': {'nombre': 'Fagúndez Campo, Antidio',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoradelasherasfernandez': {'nombre': 'De las Heras Fernández, Patricia',
                                    'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorministrodejusticia(campomoreno)': {'nombre': 'Campo Moreno, Juan Carlos',
                                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorortegasmithmolina': {'nombre': 'Ortega Smith-Molina, Francisco Javier',
                                 'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoraborraspabon': {'nombre': 'Borrás Pabón, Mireia',
                            'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoraborráspabon': {'nombre': 'Borrás Pabón, Mireia',
                             'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoraborrasegea': {'nombre': 'Borrás Pabón, Mireia',
                             'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñorarodriguezherrer': {'nombre': 'Rodríguez Herrer, María Elvira',
                                'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoramoralejagomez': {'nombre': 'Moraleja Gómez, Tristana María',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoraministradeindustriacomercioyturismo(marotoillera)': {'nombre': 'Maroto Illera, Reyes',
                                                                   'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraindustriacomercioyturismo(marotoillera)': {'nombre': 'Maroto Illera, Reyes',
                                                         'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoralopezalvarez': {'nombre': 'López Álvarez, María Teresa',
                             'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorcapdevilaiesteve': {'nombre': 'Capdevila i Esteve, Joan',
                                'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'elseñorboadellasteve': {'nombre': 'Capdevila i Esteve, Joan',
                             'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'elseñorcortesgomez': {'nombre': 'Cortés Gómez, Ismael',
                           'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoraministradeasuntosexterioresunioneuropeaycooperacion(gonzalezlaya)': {
        'nombre': 'González Laya, María Aránzazu',
        'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradeexterioresunioneuropeaycooperacion(gonzalezlaya)': {
        'nombre': 'González Laya, María Aránzazu',
        'grupo parlamentario': 'Grupo Parlamentario Socialista'}, 
    'laseñoraprohensrigo': {'nombre': 'Prohens Rigo, Margarita',
                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorbascorugeira': {'nombre': 'Bas Corugeira, Javier',
                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoracañizarespacheco': {'nombre': 'Cañizares Pacheco, Inés María',
                                 'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoracañizarespachecho': {'nombre': 'Cañizares Pacheco, Inés María',
                                  'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñorabasacoll': {'nombre': 'Bassa Coll, Montserrat',
                         'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'laseñoraministradederechossocialesyagenda2030(belarraurteaga)': {'nombre': 'Belarra Urteaga, Ione',
                                                                      'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoraministradeasuntossocialesyagenda2030(belarraurteaga)': {'nombre': 'Belarra Urteaga, Ione',
                                                                     'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoraministradederechossocialesyagenda2030(belarraureaga)': {'nombre': 'Belarra Urteaga, Ione',
                                                                     'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorgarciagomez': {'nombre': 'García Gómez, Valentín',
                           'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorabeltranvillalba': {'nombre': 'Beltrán Villalba, Ana María',
                                'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñoralmodobarbarcelo': {'nombre': 'Almodóbar Barceló, Agustín',
                                'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoraillamoladausa': {'nombre': 'Illamola Dausà, Mariona',
                              'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'laseñoracalvogomez': {'nombre': 'Calvo Gómez, Pilar',
                           'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'laseñoracalvoigomez': {'nombre': 'Calvo Gómez, Pilar',
                           'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'laseñorablanqueralcaraz': {'nombre': 'Blanquer Alcaraz, Patricia',
                                'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsuárezlamata': {'nombre': 'Suárez Lamata, Eloy',
                             'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorsuarezlamata': {'nombre': 'Suárez Lamata, Eloy',
                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorcortescarballo': {'nombre': 'Cortés Carballo, Mario',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorlopezdeuraldegarmendia': {'nombre': 'López de Uralde Garmendia, Juan Antonio',
                                      'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorlopezdeuralde': {'nombre': 'López de Uralde Garmendia, Juan Antonio',
                             'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñormansoolivar': {'nombre': 'Manso Olivar, Rubén Silvano',
                           'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñormariscalanaya': {'nombre': 'Mariscal Anaya, Guillermo',
                             'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorlamuaestañol': {'nombre': 'Lamuà Estañol, Marc',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoramuñozdalda': {'nombre': 'Muñoz Dalda, Lucía',
                           'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñoramuñozdalda': {'nombre': 'Muñoz Dalda, Lucía',
                           'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoracaballerogutierrez': {'nombre': 'Caballero Gutiérrez, Helena',
                                   'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorramosesteban': {'nombre': 'Ramos Esteban, César Joaquín',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorramirezdelrio': {'nombre': 'Ramírez del Río, José',
                             'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorrodriguezalmeida': {'nombre': 'Rodríguez Almeida, Andrés Alberto',
                                'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoranoguerasicamero': {'nombre': 'Nogueras i Camero, Míriam',
                                'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'elseñorrodriguezsalas': {'nombre': 'Rodríguez Salas, José Antonio',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoracanteradecastro': {'nombre': 'Cantera de Castro, Zaida',
                                'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorarosmartinez': {'nombre': 'Ros Martínez, Susana',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoranevadodelcampo': {'nombre': 'Nevado del Campo, María Magdalena',
                               'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorrosetyfernandezdecastro': {'nombre': 'Rosety Fernández de Castro, Agustín',
                                       'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorgillazaro': {'nombre': 'Gil Lázaro, Ignacio',
                         'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorgillázaro': {'nombre': 'Gil Lázaro, Ignacio',
                         'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorgutierrezvivas': {'nombre': 'Gutiérrez Vivas, Miguel Ángel',
                              'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'elseñorgutiérrezvivas': {'nombre': 'Gutiérrez Vivas, Miguel Ángel',
                               'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'laseñorapeñacamarero': {'nombre': 'Peña Camarero, Esther',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorhonrubiahurtado': {'nombre': 'Honrubia Hurtado, Pedro Antonio',
                               'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñoraizcorbetorra': {'nombre': 'Aizcorbe Torra, Juan José',
                             'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoragarciapelayojurado': {'nombre': 'García-Pelayo Jurado, María José',
                                   'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoraguaitaesteruelas': {'nombre': 'Guaita Esteruelas, Sandra',
                                 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñoralvarezigarcia': {'nombre': 'Álvarez i García, Gerard',
                              'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'elseñorgutierrezdiazdeotazu': {'nombre': 'Gutiérrez Díaz de Otazu, Fernando Adolfo',
                                    'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoramendezmonasterio': {'nombre': 'Méndez Monasterio, Lourdes',
                                 'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoraministradeigualdad(monterogil)': {'nombre': 'Montero Gil, Irene María',
                                               'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorsanchezgarcia': {'nombre': 'Sánchez García, José María',
                             'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoraalvarezfanjul': {'nombre': 'Álvarez Fanjul, Beatriz',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorpaniaguanuñez': {'nombre': 'Paniagua Núñez, Miguel Ángel',
                             'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorpaniagauanuñez': {'nombre': 'Paniagua Núñez, Miguel Ángel',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoramontesinosdemiguel': {'nombre': 'Montesinos de Miguel, Macarena',
                                   'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorsanchoguardia': {'nombre': 'Sancho Guardia, Alfredo',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorministrodeuniversidades(castellsolivan)': {'nombre': 'Castells Oliván, Manuel',
                                                       'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorministrodeuniversidades(subiratshumet)': {'nombre': 'Subirats Humet, Joan',
                                                      'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorgagobugarin': {'nombre': 'Gago Bugarín, Diego',
                           'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorministrodepoliticaterritorialyfuncionpublica(icetaillorens)': {
        'nombre': 'Iceta i Llorens, Miquel', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorministrodeculturaydeporte(icetaillorens)': {'nombre': 'Iceta i Llorens, Miquel',
                                                        'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorgonzalezcaballero': {'nombre': 'González Caballero, Miguel Ángel',
                                 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorcaballerogonzalez': {'nombre': 'González Caballero, Miguel Ángel',
                                 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorpedreñomolina': {'nombre': 'Pedreño Molina, Juan Luis',
                             'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoraredondocalvillo': {'nombre': 'Redondo Calvillo, María de la O',
                                'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorcalvoliste': {'nombre': 'Calvo Liste, Pablo Juan',
                          'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoragarciarodriguez': {'nombre': 'García Rodríguez, Alicia',
                                'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorarandavargas': {'nombre': 'Aranda Vargas, Francisco',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsantamariaruiz': {'nombre': 'Santamaría Ruiz, Luis',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorsantamaria': {'nombre': 'Santamaría Ruiz, Luis',
                          'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñoralcarazmartos': {'nombre': 'Alcaraz Martos, Francisco José',
                             'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorpagesimasso': {'nombre': 'Pagès i Massó, Josep',
                           'grupo parlamentario': 'Grupo Parlamentario Plural'},
    'laseñoravelardegomez': {'nombre': 'Velarde Gómez, Martina',
                             'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñoruriartetorrealday': {'nombre': 'Uriarte Torrealday, Roberto',
                                 'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñoruriarteardialay': {'nombre': 'Uriarte Torrealday, Roberto',
                               'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñordeolanovela': {'nombre': 'Olano Vela, Jaime Eduardo de',
                           'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorolanovela': {'nombre': 'Olano Vela, Jaime Eduardo de',
                         'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoracarrillodelosreyes': {'nombre': 'Carrillo de los Reyes, Beatriz Micaela',
                                   'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorserradapariente': {'nombre': 'Serrada Pariente, David',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorrenaumartinez': {'nombre': 'Renau Martínez, Germán',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsegurajust': {'nombre': 'Segura Just, Juan Carlos',
                          'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñornavalpotrogomez': {'nombre': 'Navalpotro Gómez, Julio',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsaezalonsomuñumer': {'nombre': 'Sáez Alonso-Muñumer, Pablo',
                                 'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorrequenaruiz': {'nombre': 'Requena Ruiz, Juan Diego',
                           'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoratizonvazquez': {'nombre': 'Tizón Vázquez, Uxía',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñornavarrolopez': {'nombre': 'Navarro López, Pedro',
                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorzaragozaalonso': {'nombre': 'Zaragoza Alonso, José',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoramarradominguez': {'nombre': 'Marra Domínguez, María Ángeles',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravelascomorillo': {'nombre': 'Velasco Morillo, Elvira',
                               'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoracastillolopez': {'nombre': 'Castillo López, Elena',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorlopezbasvalero': {'nombre': 'López-Bas Valero, Juan Ignacio',
                              'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'elseñormeijoncouselo': {'nombre': 'Meijón Couselo, Guillermo Antonio',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorjimenezrevuelta': {'nombre': 'Jiménez Revuelta, Rodrigo',
                               'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoramarquezguerrero': {'nombre': 'Márquez Guerrero, María',
                                'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoramárquezguerrero': {'nombre': 'Márquez Guerrero, María',
                                 'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorasartacueva': {'nombre': 'Asarta Cuevas, Alberto',
                           'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorasartacuevas': {'nombre': 'Asarta Cuevas, Alberto',
                            'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñorarepresentantedelparlamentodemurcia(marinmartinez)': {'nombre': 'Marín Martínez, María',
                                                                  'grupo parlamentario': 'Parlamento de Murcia'},
    'elseñorortizgalvan': {'nombre': 'Ortiz Galván, José',
                           'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorortizgalván': {'nombre': 'Ortiz Galván, José',
                           'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñorabravobarco': {'nombre': 'Bravo Barco, Eva',
                           'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorrepresentantedelaasamblearegionaldemurcia(segadomartinez)': {
        'nombre': 'Segado Martínez, Joaquín', 'grupo parlamentario': 'Parlamento de Murcia'},
    'elseñorrepresentantedelaasamblearegionaldemurcia(conesaalcaraz)': {'nombre': 'Conesa Alcaraz, Diego',
                                                                        'grupo parlamentario': 'Parlamento de Murcia'},
    'elseñorrequejonovoa': {'nombre': 'Requejo Novoa, Pedro Jesús',
                            'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorrequejotal': {'nombre': 'Requejo Novoa, Pedro Jesús',
                          'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorhispaniglesiasdeussel': {'nombre': 'Hispán Iglesias de Ussel, Pablo',
                                     'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoraborregocortes': {'nombre': 'Borrego Cortés, Isabel María',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorrobleslopez': {'nombre': 'Robles López, Joaquín',
                           'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoracruzguzmangarcia': {'nombre': 'Cruz-Guzmán García, María Soledad',
                                 'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoracruzguzmángarcia': {'nombre': 'Cruz-Guzmán García, María Soledad',
                                  'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñortaibomonelos': {'nombre': 'Taibo Monelos, Diego',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoragarciapuig': {'nombre': 'García Puig, María del Mar',
                           'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñorabañosruiz': {'nombre': 'Baños Ruiz, Carmen',
                          'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraestebancalonje': {'nombre': 'Esteban Calonje, Cristina Alicia',
                               'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorsotoburillo': {'nombre': 'Soto Burillo, Juan Luis',
                           'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsteegmannolmedillas': {'nombre': 'Steegmann Olmedillas, Juan Luis',
                                   'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorsteegmanolmedillas': {'nombre': 'Steegmann Olmedillas, Juan Luis',
                                   'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoraministradeeducacionyformacionprofesional(celaadieguez)': {'nombre': 'Celaá Diéguez, Isabel',
                                                                       'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraperezmerino': {'nombre': 'Pérez Merino, María Mercedes',
                            'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorbustamantemartin': {'nombre': 'Bustamante Martín, Miguel Ángel',
                                'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorbustamantemartinez': {'nombre': 'Bustamante Martín, Miguel Ángel',
                                  'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoravidalsaez': {'nombre': 'Vidal Sáez, Aina',
                          'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoravidazsaez': {'nombre': 'Vidal Sáez, Aina',
                          'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorjerezjuan': {'nombre': 'Jerez Juan, Miguel Ángel',
                         'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñorafernandezbeneitez': {'nombre': 'Fernández Benéitez, Andrea',
                                  'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorafernandezbenéitez': {'nombre': 'Fernández Benéitez, Andrea',
                                  'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorvicenteviondi': {'nombre': 'Vicente Viondi, Daniel',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorcabezoncasas': {'nombre': 'Cabezón Casas, Tomás',
                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñormovellanlombilla': {'nombre': 'Movellán Lombilla, Diego',
                                'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoraruizsolas': {'nombre': 'Ruiz Solás, María de la Cabeza',
                          'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoraruizsolás': {'nombre': 'Ruiz Solás, María de la Cabeza',
                           'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorcallejascano': {'nombre': 'Callejas Cano, Juan Antonio',
                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorgonzalezramos': {'nombre': 'González Ramos, Manuel Gabriel',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraguinartmoreno': {'nombre': 'Guinart Moreno, Lídia',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraguijarroceballos': {'nombre': 'Guijarro Ceballos, María',
                                 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsanchezdelreal': {'nombre': 'Sánchez del Real, Víctor Manuel',
                              'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoraprietonieto': {'nombre': 'Prieto Nieto, Ana',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorramosrodriguez': {'nombre': 'Ramos Rodríguez, José Luis',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraramonutrabo': {'nombre': 'Ramón Utrabo, Elvira',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñormerinomartinez': {'nombre': 'Merino Martínez, Javier',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñormariklose': {'nombre': 'Marí Klose, Pau',
                         'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorgonzalezterol': {'nombre': 'González Terol, Antonio',
                             'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñorabuenocampanario': {'nombre': 'Bueno Campanario, Eva Patricia',
                                'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorecheniquerobba': {'nombre': 'Echenique Robba, Pablo',
                              'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñorajoverdiaz': {'nombre': 'Jover Díaz, Antonia',
                          'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorjoverdiaz': {'nombre': 'Jover Díaz, Antonia',
                          'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorsanchezperez': {'nombre': 'Sánchez Pérez, César',
                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorgestosodemiguel': {'nombre': 'Gestoso de Miguel, Luis',
                               'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñoracevesgalindo': {'nombre': 'Aceves Galindo, José Luis',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorafernandezcasero': {'nombre': 'Fernández Casero, Ana Belén',
                                'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorafanecalopez': {'nombre': 'Faneca López, María Luisa',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoramartinezseijo': {'nombre': 'Martínez Seijo, María Luz',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorarribasmaroto': {'nombre': 'Arribas Maroto, Manuel',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraaraujomorales': {'nombre': 'Araujo Morales, Gemma',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoramartinllaguno': {'nombre': 'Martín Llaguno, Marta',
                              'grupo parlamentario': 'Grupo Parlamentario Ciudadanos'},
    'elseñorortegadominguez': {'nombre': 'Ortega Domínguez, José Ramón',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoralopezsomoza': {'nombre': 'López Somoza, Gema',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorponssampietro': {'nombre': 'Pons Sampietro, Pere Joan',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorazorinsalar': {'nombre': 'Azorín Salar, Lázaro',
                           'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraorialopez': {'nombre': 'Oria López, María Inmaculada',
                          'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsenderosoraa': {'nombre': 'Senderos Oraá, Daniel',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorvegaarias': {'nombre': 'Vega Arias, Rubén Darío',
                         'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñornuetpujals': {'nombre': 'Nuet Pujals, Joan Josep',
                          'grupo parlamentario': 'Grupo Parlamentario Republicano'},
    'elseñorhurtadozurera': {'nombre': 'Hurtado Zurera, Antonio',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorhurtazozurera': {'nombre': 'Hurtado Zurera, Antonio',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorministrodelapresidenciarelacionesconlascortesymemoriademocratica(bolañosgarcia)': {
        'nombre': 'Bolaños García, Félix', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorministrodelapresidenciarelacionesconlascortesymemoriahistorica(bolañosgarcia)': {
        'nombre': 'Bolaños García, Félix', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraromerosanchez': {'nombre': 'Romero Sánchez, Rosa María',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorramirezcarner': {'nombre': 'Ramírez Carner, Arnau',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorrepresentantedelparlamentodenavarra(azconamolinet)': {'nombre': 'Azcona Molinet, Pablo',
                                                                  'grupo parlamentario': 'Parlamento de Navarra'},
    'laseñorarepresentantedelparlamentodenavarra(desimoncaballero)': {
        'nombre': 'De Simón Caballero, Marisa', 'grupo parlamentario': 'Parlamento de Navarra'},
    'elseñorrepresentantedelparlamentodenavarra(builgarcia)': {'nombre': 'Buil García, Mikel',
                                                               'grupo parlamentario': 'Parlamento de Navarra'},
    'elseñorcerdanleon': {'nombre': 'Cerdán León, Santos',
                          'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorazuritaexposito': {'nombre': 'Zurita Expósito, Ana María',
                               'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñorazuritaexpoxito': {'nombre': 'Zurita Expósito, Ana María',
                               'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoraministradepoliticaterritorial(rodriguezgarcia)': {'nombre': 'Rodríguez García, Isabel',
                                                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradepoliticaterritorialyportavozdelgobierno(rodriguezgarcia)': {
        'nombre': 'Rodríguez García, Isabel', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradepoliticaterritorialyfuncionpublica(rodriguezgarcia)': {
        'nombre': 'Rodríguez García, Isabel', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradejusticia(llopcuenca)': {'nombre': 'Llop Cuenca, Pilar',
                                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorgarciamoris': {'nombre': 'García Morís, Roberto',
                           'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraberjavega': {'nombre': 'Berja Vega, Laura',
                          'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoragazquezcollado': {'nombre': 'Gázquez Collado, Paloma',
                               'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoraanguloromero': {'nombre': 'Angulo Romero, María Teresa',
                             'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorbetoretcoll': {'nombre': 'Betoret Coll, Vicente',
                           'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoramoneodiez': {'nombre': 'Moneo Díez, María Sandra',
                          'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoraministradeeducacionyformacionprofesional(alegriacontinente)': {
        'nombre': 'Alegría Continente, Pilar', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñordefensordelpuebloenfuncionesyadjuntoprimero(fernandezmarugan)': {
        'nombre': 'Fernández Marugán, Francisco',
        'grupo parlamentario': 'Defensor del Pueblo en Funciones'},
    'elseñorconstenlacarbon': {'nombre': 'Constenla Carbón, Juan Manuel',
                               'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorruiznavarro': {'nombre': 'Ruiz Navarro, Eduardo Luis',
                           'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorelorriagapisarik': {'nombre': 'Elorriaga Pisarik, Gabriel',
                                'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorsanchezescobar': {'nombre': 'Sánchez Escobar, Mariano',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñordelgadoarce': {'nombre': 'Delgado Arce, Celso Luis',
                           'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoraandresañon': {'nombre': 'Andrés Añón, Carmen',
                           'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorasanchezjodar': {'nombre': 'Sánchez Jódar, Marisol',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorasánchezjodar': {'nombre': 'Sánchez Jódar, Marisol',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraferrertesoro': {'nombre': 'Ferrer Tesoro, Sonia',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñormariscalzabala': {'nombre': 'Mariscal Zabala, Manuel',
                              'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñorajaramoreno': {'nombre': 'Jara Moreno, Mercedes',
                           'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorsalvaverd': {'nombre': 'Salvá Verd, Antonio',
                         'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorministrodeasuntosexterioresunioneuropeaycooperacion(albaresbueno)': {
        'nombre': 'Albares Bueno, José Manuel', 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorministrodeasuntosexterioresunioneuropeaycooperacion': {'nombre': 'Albares Bueno, José Manuel',
                                                                    'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorariolobosregadera': {'nombre': 'Riolobos Regadera, Carmen',
                                 'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñorajiménezbecerrilbarrio': {'nombre': 'Jiménez-Becerril Barrio, María Teresa',
                                       'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñorajimenezbecerrilbarrio': {'nombre': 'Jiménez-Becerril Barrio, María Teresa',
                                      'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorruizicarbonell': {'nombre': 'Ruiz i Carbonell, Joan',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorgutiérrezprieto': {'nombre': 'Gutiérrez Prieto, Sergio',
                                'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorgutierrezprieto': {'nombre': 'Gutiérrez Prieto, Sergio',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoragonzalezpérez': {'nombre': 'González Pérez, Ariagona',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoragonzalezperez': {'nombre': 'González Pérez, Ariagona',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñordelgadoramos': {'nombre': 'Delgado Ramos, Juan Antonio',
                            'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorledesmamartin': {'nombre': 'Ledesma Martín, Sebastián Jesús',
                             'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoragarciachavarria': {'nombre': 'García Chavarría, María Montserrat',
                                'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoratoscanodebalbin': {'nombre': 'Toscano de Balbín, Carla',
                                'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorcontreraspelaez': {'nombre': 'Contreras Peláez, Francisco José',
                               'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorcontreraspeláez': {'nombre': 'Contreras Peláez, Francisco José',
                                'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoranarvaezbandera': {'nombre': 'Narváez Bandera, María Dolores',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoranavaezbandera': {'nombre': 'Narváez Bandera, María Dolores',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorherrerobono': {'nombre': 'Herrero Bono, José Alberto',
                           'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorherrerabono': {'nombre': 'Herrero Bono, José Alberto',
                           'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñorapadillaruiz': {'nombre': 'Padilla Ruiz, Esther',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoracanalesduque': {'nombre': 'Canales Duque, Mariana de Gracia',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoratriasgil': {'nombre': 'Trías Gil, Georgina',
                         'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorcarazohermoso': {'nombre': 'Carazo Hermoso, Eduardo',
                             'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoragarcialopez': {'nombre': 'García López, Maribel',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñoralfonsocendon': {'nombre': 'Alfonso Cendón, Javier',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoranasarreoliva': {'nombre': 'Nasarre Oliva, Begoña',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorantoncacho': {'nombre': 'Antón Cacho, Javier',
                          'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorantonchacho': {'nombre': 'Antón Cacho, Javier',
                           'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorperezabellas': {'nombre': 'Pérez Abellás, Adolfo',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñortiradoochoa': {'nombre': 'Tirado Ochoa, Vicente',
                           'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoralimacid': {'nombre': 'Lima Cid, Fuensanta',
                        'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoracrespinrubio': {'nombre': 'Crespín Rubio, Rafaela',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsolermur': {'nombre': 'Soler Mur, Alejandro',
                        'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradecienciaeinnovacion(morantripoll)': {'nombre': 'Morant Ripoll, Diana',
                                                           'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraandresbarea': {'nombre': 'Andrés Barea, Josefa',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñoralonsoperez': {'nombre': 'Alonso Pérez, José Ángel',
                           'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñordelvallerodriguez': {'nombre': 'Del Valle Rodríguez, Emilio Jesús',
                                 'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñordioufdioh': {'nombre': 'Diouf Dioh, Luc Andre',
                         'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorfernandezrios': {'nombre': 'Fernández Ríos, Tomás',
                             'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorfernándezrios': {'nombre': 'Fernández Ríos, Tomás',
                              'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñormateuisturiz': {'nombre': 'Mateu Istúriz, Jaime Miguel',
                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorclavelllopez': {'nombre': 'Clavell López, Óscar',
                            'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorgamazomico': {'nombre': 'Gamazo Micó, Óscar',
                          'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorserranomartinez': {'nombre': 'Serrano Martínez, Juan Francisco',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñortarnoblanco': {'nombre': 'Tarno Blanco, Ricardo',
                           'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorduranperalta': {'nombre': 'Durán Peralta, José Carlos',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoravilchesruiz': {'nombre': 'Vilches Ruiz, María Luisa',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorlopezcano': {'nombre': 'López Cano, Ignacio',
                         'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorrepresentantedelparlamentodeandalucia(ruizortiz)': {'nombre': 'Ruiz Ortiz, Miguel Ángel',
                                                                'grupo parlamentario': 'Parlamento de Andalucía'},
    'laseñorarepresentantedelparlamentodeandalucia(marquezromero)': {'nombre': 'Márquez Romero, María',
                                                                     'grupo parlamentario': 'Parlamento de Andalucía'},
    'laseñorarepresentantedelparlamentodeandalucia(sanchezmuñoz)': {
        'nombre': 'Sánchez Muñoz, María del Mar', 'grupo parlamentario': 'Parlamento de Andalucía'},
    'laseñorajimenezlinuesa': {'nombre': 'Jiménez Linuesa, Beatriz',
                               'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñorapastorjulian': {'nombre': 'Pastor Julián, Ana María',
                             'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorpostigoquintana': {'nombre': 'Postigo Quintana, Jesús',
                               'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñorarepresentantedelajuntageneraldelprincipadodeasturias(vallinadelanoval)': {
        'nombre': 'Vallina de la Noval, Ángela',
        'grupo parlamentario': 'Junta General del Principado de Asturias'},
    'laseñorarepresentantedelajuntageneraldelprincipadodeasturias(malladadecastro)': {
        'nombre': 'Mallada de Castro, María Teresa',
        'grupo parlamentario': 'Junta General del Principado de Asturias'},
    'laseñorarepresentantedelajuntageneraldelprincipadodeasturias(carcedogarcia)': {
        'nombre': 'Carcedo García, María Dolores',
        'grupo parlamentario': 'Junta General del Principado de Asturias'},
    'laseñoragonzalezguinda': {'nombre': 'González Guinda, María del Carmen',
                               'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorfernandezlomanagutierrez': {'nombre': 'Fernández-Lomana Gutiérrez, Rafael',
                                        'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoravaleriocordero': {'nombre': 'Valerio Cordero, Magdalena',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñordiazrodriguez': {'nombre': 'Díaz Rodríguez, José Carlos',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoralopezzamora': {'nombre': 'López Zamora, Cristina',
                            'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorsanchoiñiguez': {'nombre': 'Sancho Íñiguez, Herminio Rufino',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñormanglanoalbacar': {'nombre': 'Manglano Albacar, Percival',
                               'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñordefensordelpueblo(gabilondopujol)': {'nombre': 'Gabilondo Pujol, Ángel',
                                                 'grupo parlamentario': 'Defensor del Pueblo'},
    'elseñorgonzalezmuñoz': {'nombre': 'González Muñoz, Ángel Luis',
                             'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorrepresentantedelparlamentodecantabria(fernandezgarcia)': {'nombre': 'Fernández García, Íñigo',
                                                                      'grupo parlamentario': 'Parlamento de Cantabria'},
    'elseñorrepresentantedelparlamentodecantabria(alvarezpalleiro)': {'nombre': 'Álvarez Palleiro, Félix',
                                                                      'grupo parlamentario': 'Parlamento de Cantabria'},
    'elseñorrepresentantedelparlamentodecantabria(hernandogarcia)': {
        'nombre': 'Hernando García, Pedro José', 'grupo parlamentario': 'Parlamento de Cantabria'},
    'laseñoraperezdiaz': {'nombre': 'Pérez Díaz, María Auxiliadora',
                          'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorfernandezrocasuarez': {'nombre': 'Fernández-Roca Suárez, Carlos Hugo',
                                   'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'elseñorzambranogarciaraez': {'nombre': 'Zambrano García-Raez, Carlos José',
                                  'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñoraalonsosuarez': {'nombre': 'Alonso Suárez, María Olga',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorromerohernandez': {'nombre': 'Romero Hernández, Carmelo',
                               'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoravillagrasaquero': {'nombre': 'Villagrasa Quero, Noemí',
                                'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorbuenopinto': {'nombre': 'Bueno Pinto, José Luis',
                          'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'laseñoramarcosdominguez': {'nombre': 'Marcos Domínguez, Pilar',
                                'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñormontesinosaguayo': {'nombre': 'Montesinos Aguayo, Pablo',
                                'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñormirallesmartin': {'nombre': 'Miralles Martín, Onofre',
                              'grupo parlamentario': 'Grupo Parlamentario VOX'},
    'laseñorafernandezperez': {'nombre': 'Fernández Pérez, María',
                               'grupo parlamentario': 'Grupo Parlamentario Mixto'},
    'laseñoraperezrecuerda': {'nombre': 'Pérez Recuerda, Isabel Gema',
                              'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorromanicantera': {'nombre': 'Romaní Cantera, José Ignacio',
                             'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'elseñorrepresentantedelascortesdearagon(aliagalopez)': {'nombre': 'Aliaga López, Arturo',
                                                             'grupo parlamentario': 'Parlamento de Aragón'},
    'laseñorarepresentantedelascortesdearagon(vaqueroperianez)': {
        'nombre': 'Vaquero Periánez, Maria del Mar', 'grupo parlamentario': 'Parlamento de Aragón'},
    'laseñorarepresentantedelascortesdearagon(garciagonzalez)': {'nombre': 'García González, Beatriz',
                                                                 'grupo parlamentario': 'Parlamento de Aragón'},
    'elseñorcerqueirogonzalez': {'nombre': 'Cerqueiro González, Javier',
                                 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorabotellagomez': {'nombre': 'Botella Gómez, Ana María',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorarepresentantedelaasambleadelacomunidaddemadrid(negrokonrad)': {
        'nombre': 'Negro Konrad, Almudena', 'grupo parlamentario': 'Asamblea de Madrid'},
    'elseñorrepresentantedelaasambleadelacomunidaddemadrid(gutierrezdecabiedeshidalgodecaviedes)': {
        'nombre': 'Gutiérrez de Cabiedes Hidalgo de Caviedes, Pablo',
        'grupo parlamentario': 'Asamblea de Madrid'},
    'elseñorzapatasimon': {'nombre': 'Zapata Simón, Aurelio',
                           'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoraministradeindustriacomercioyturismo(reyesmaroto)': {'nombre': 'Reyes Maroto, María',
                                                                  'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorgarcianieto': {'nombre': 'García Nieto, Ezequiel',
                           'grupo parlamentario': 'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común'},
    'elseñorcuatrecasasasua': {'nombre': 'Cuatrecasas Asua, Juan',
                               'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorministrodeculturaydeporte(rodriguezuribes)': {'nombre': 'Rodríguez Uribes, José Manuel',
                                                          'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorquintanillanavarro': {'nombre': 'Quintanilla Navarro, Miguel Ángel',
                                  'grupo parlamentario': 'Grupo Parlamentario Popular en el Congreso'},
    'laseñoracarcedoroces': {'nombre': 'Carcedo Roces, María Luisa',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoracancelarodriguez': {'nombre': 'Cancela Rodríguez, Pilar',
                                 'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'elseñorministrodecienciaeinnovacion(duqueduque)': {'nombre': 'Duque Duque, Pedro',
                                                        'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorasumelzojordan': {'nombre': 'Sumelzo Jordán, Susana',
                              'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorahernanzcosta': {'nombre': 'Hernanz Costa, Sofía',
                             'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñorarepresentantedelparlamentodecantabria(coboperez)': {'nombre': 'Cobo Pérez, Íñigo',
                                                                 'grupo parlamentario': 'Parlamento de Cantabria'},
    'elseñorrepresentantedelparlamentodecataluña(pedretisantos)': {'nombre': 'Pedret i Santos, Ferran',
                                                                   'grupo parlamentario': 'Parlamento de Cataluña'},
    'laseñorarepresentantedelparlamentodecataluña(albiachsatorres)': {
        'nombre': 'Albiach Satorres, Jéssica', 'grupo parlamentario': 'Parlamento de Cataluña'},
    'elseñorrepresentantedelparlamentodecataluña(pujolibonell)': {'nombre': 'Pujol i Bonell, Eduard',
                                                                  'grupo parlamentario': 'Parlamento de Cataluña'},
    'elseñorsahuquillogarcia': {'nombre': 'Sahuquillo García, Luis Carlos',
                                'grupo parlamentario': 'Grupo Parlamentario Socialista'},
    'laseñoragonzalezlaso':
    {'nombre':'González Laso, Natividad', 'grupo parlamentario':'Grupo Parlamentario Socialista'},
    'elseñorrepresentantedelaasamblearegionaldemurcia(molinagallardo)': 
    {'nombre': 'Molina Gallardo, Juan José', 'grupo parlamentario': 'Parlamento de Murcia'},
    'laseñorarepresentantedelparlamentodegalicia(pontonmondelo)': 
    {'nombre': 'Pontón Mondelo, Ana Belén', 'grupo parlamentario': 'Parlamento de Galicia'},
    'elseñorrepresentantedelparlamentodegalicia(caballeromiguez)': 
    {'nombre': 'Caballero Míguez, Gonzalo', 'grupo parlamentario': 'Parlamento de Galicia'},
    'elseñorrepresentantedelparlamentodegalicia(calvopouso)': 
    {'nombre': 'Calvo Pouso, Diego', 'grupo parlamentario': 'Parlamento de Galicia'},
    'elseñorrepresentantedelparlamentodelasillesbalears(ferraterrasa)': 
    {'nombre': 'Ferrà Terrassa, Joan', 'grupo parlamentario': 'Parlamento de Illes Balears'},
    'elseñorrepresentantedelparlamentodelasillesbalears(sagrerasballester)': 
    {'nombre': 'Sagreras Ballester, Sebastián', 'grupo parlamentario': 'Parlamento de Illes Balears'},
    'elseñorrepresentantedelparlamentodelasillesballears(jimenezfernandez)': 
    {'nombre': 'Jiménez Fernández, Pablo Jesús', 'grupo parlamentario': 'Parlamento de Illes Balears'},
    'elseñorrepresentantedelparlamentodelasillesbalears(meliaiques)': 
    {'nombre': 'Melià i Ques, Josep', 'grupo parlamentario': 'Parlamento de Illes Balears'},
    'elseñorrepresentantedelparlamentodelasillesbalears(casanovapeiro)': 
    {'nombre': 'Casanova Peiró, Enrique', 'grupo parlamentario': 'Parlamento de Illes Balears'},
    'elseñorrepresentantedelparlamentodelasillesbalears(maritorres)': 
    {'nombre': 'Marí Torres, Vicente Alejandro', 'grupo parlamentario': 'Parlamento de Illes Balears'},
    'laseñorarodriguezgomez':
    {'nombre':'Rodríguez Gómez, María del Pilar', 'grupo parlamentario':'Grupo Parlamentario Socialista'},
    'elseñormatarisaez':
    {'nombre':'Matarí Sáez, Juan José', 'grupo parlamentario':'Grupo Parlamentario Popular en el Congreso'},
    'elseñorrepresentantedelparlamentodelasillesbalears(dalmaudemata)': 
    {'nombre': 'Dalmau de Mata, Juli ', 'grupo parlamentario': 'Parlamento de Illes Balears'},
    'elseñorrepresentantedelparlamentodelasillesbalears(castellsbaro)': 
    {'nombre': 'Castells Baró, Josep', 'grupo parlamentario': 'Parlamento de Illes Balears'},
    'elseñorrepresentantedelparlamentodelasillesbalears(lafuentemir)': 
    {'nombre': 'Lafuente Mir, Juan Manuel', 'grupo parlamentario': 'Parlamento de Illes Balears'},   
    'laseñorarepresentantedelparlamentovasco(blancodeangulo)': 
    {'nombre': 'Blanco de Angulo, Eba', 'grupo parlamentario': 'Parlamento de Pais Vasco'},
    'laseñorarepresentantedelparlamentovasco(sanchezmartin)': 
    {'nombre': 'Sánchez Martín, Gloria', 'grupo parlamentario': 'Parlamento de Pais Vasco'},
    'laseñorarepresentantedelparlamentovasco(ipiñazarmiranda)': 
    {'nombre': 'Ipiñazar Miranda, Maitane', 'grupo parlamentario': 'Parlamento de Pais Vasco'},
    'laseñorasevaruiz':
    {'nombre':'Seva Ruiz, Yolanda', 'grupo parlamentario':'Grupo Parlamentario Socialista'},
    'elseñorzamarronmoreno':
    {'nombre':'Zamarrón Moreno, Agustín', 'grupo parlamentario':'Grupo Parlamentario Socialista'},
    'elseñorrepresentantedelparlamentodelarioja(diazmarin)': 
    {'nombre': 'Díaz Marín, Raúl', 'grupo parlamentario': 'Parlamento de La Rioja'},
    'elseñorrepresentantedelparlamentodelarioja(baenapedrosa)': 
    {'nombre': 'Baena Pedrosa, Pablo', 'grupo parlamentario': 'Parlamento de La Rioja'},
    'elseñorrepresentantedelparlamentodelarioja(garridomartinez)': 
    {'nombre': 'Garrido Martínez, Jesús Ángel', 'grupo parlamentario': 'Parlamento de La Rioja'},
    'elseñorrepresentantedelajuntageneraldelprincipadodeasturias(zapicogonzalez)': 
    {'nombre': 'Zapico González, Ovidio', 'grupo parlamentario': 'Parlamento del principado de Asturias'},
    'elseñorrepresentantedelajuntageneraldelprincipadodeasturias(mariripa)': 
    {'nombre': 'Marí Ripa, Daniel', 'grupo parlamentario': 'Parlamento del principado de Asturias'},
}


In [369]:
print(datetime.now())
dialogos_completos = []
for documento in [arch for arch in ls() if arch.endswith('.PDF')]:
    if documento >= r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-1.PDF':
        documentos_sin_dialogos = [
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-5.PDF',
    r'C:\Users\jmelgar\Congreso\documentos\DSCD-14-PL-107.PDF',
]
        reader = PdfReader(documento)
        paginas, fecha_proceso = extrae_paginas(reader)
        paginas_procesadas = procesa_paginas(paginas, documento)
        paginas_limpias = limpia_paginas(paginas_procesadas)
        texto = extrae_texto(paginas_limpias, documento)
        dialogos_agrupados = agrupa_dialogos(texto)
        dialogos = elimina_puntuacion(dialogos_agrupados,fecha_proceso)

        if len(dialogos) == 0 and documento not in documentos_sin_dialogos:
            print(documento)
            a = 0
            b = 7/a

        hablantes_faltantes = []
        hablantes = []
        for i in dialogos:
            hablantes.append(i['hablante_transformado'])
        for el in list(set(hablantes)):
            try:
                a = dict_hablantes[el]
            except:
                hablantes_faltantes.append(el)

        
        if len(hablantes_faltantes) > 0 and documento not in documentos_sin_dialogos:
            print(documento)
            a = 0
            b = 7/a
            
        if documento not in documentos_sin_dialogos:
            for linea in dialogos:
                dialogos_completos.append({'hablante': dict_hablantes[linea['hablante_transformado']],
                                           'discurso': linea['discurso'],
                                           'fecha_proceso': linea['fecha_proceso']
                                          })
        break
        
print(datetime.now())

2023-02-03 12:09:24.315145
2023-02-03 12:09:25.018025


In [ ]:
jjjjj

In [380]:
with open('dialogos.pickle', "rb") as f:
    dialogos_completos = pickle.load(f)

In [370]:
print(len(dialogos_completos))
palabras = []
for el in dialogos_completos:
    palabras.extend(el['discurso'].split(' '))
print(len(palabras))

84
30932


In [371]:
def obtener_metrica_szigriszt_pazos(discurso):
    '''
    P	estilo	tipo de publicación	estudios
    0 a 15	muy difícil	científica, filosófica	titulados universitarios
    16 a 35	árido	pedagógica, técnica	selectividad y estudios universitarios
    36 a 50	bastante difícil	literatura y divulgación	cursos secundarios
    51 a 65	normal	Los media	popular
    66 a 75	bastante fácil	novela, revista	12 años
    76 a 85	fácil	para quioscos	11 años
    86 a 100	muy fácil	cómics, tebeos y viñetas	6 a 10 años
    '''
    return textstat.szigriszt_pazos(re.sub(r'\(.*?\)', '', discurso).strip())

In [372]:
def obtener_metrica_flesch_reading_ease(discurso):
    '''
    Score 	Difficulty
    90-100 	Very Easy
    80-89 	Easy
    70-79 	Fairly Easy
    60-69 	Standard
    50-59 	Fairly Difficult
    30-49 	Difficult
    0-29 	Very Confusing
    '''
    return textstat.flesch_reading_ease(re.sub(r'\(.*?\)', '', discurso).strip())

In [373]:
dialogos_completos_metricas = []
for elem in dialogos_completos:
    elem['szigriszt_pazos'] = obtener_metrica_szigriszt_pazos(elem['discurso'])
    elem['flesch_reading_ease'] = obtener_metrica_flesch_reading_ease(elem['discurso'])
    dialogos_completos_metricas.append(elem)
dialogos_completos = dialogos_completos_metricas

In [16]:
jjjj

NameError: name 'jjjj' is not defined

In [374]:
nlp = spacy.load('es_core_news_sm')
df_lemmas= pd.DataFrame()
for num, elem in enumerate(dialogos_completos):
    doc = nlp(remove_punctuation(elem['discurso'].strip()))
    lemmas = [[tok.lemma_,
               tok.orth_,
               elem['hablante']['nombre'],
               elem['hablante']['grupo parlamentario'],
               elem['fecha_proceso'],
               elem['szigriszt_pazos'],
               elem['flesch_reading_ease'],
               tok.pos_,
               num,
               i
               ] for i, tok in enumerate(doc) if (tok.pos_ not in ['PRON', 'ADP', 'DET', 'SCONJ', 'CCONJ']) &
              (not tok.is_punct | tok.is_stop)]

    
    df_temp = pd.DataFrame(lemmas, columns=['lemma', 'palabra', 'hablante', 'grupo parlamentario', 'fecha', 'szigriszt_pazos','flesch_reading_ease', 'tipo palabra', 'numero_dicurso', 'orden'])
    df_lemmas = pd.concat([df_lemmas, df_temp], axis=0)

In [375]:
df_lemmas

,lemma,palabra,hablante,grupo parlamentario,fecha,szigriszt_pazos,flesch_reading_ease,tipo palabra,numero_dicurso,orden
0,Señor,Señor,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso,2020-02-19,-79.60,-79.01,PROPN,0,0
1,Sánchez,Sánchez,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso,2020-02-19,-79.60,-79.01,PROPN,0,1
2,bloquear,bloquea,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso,2020-02-19,-79.60,-79.01,VERB,0,3
3,ayer,ayer,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso,2020-02-19,-79.60,-79.01,ADV,0,9
4,tener,tuvimos,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso,2020-02-19,-79.60,-79.01,VERB,0,10
...,...,...,...,...,...,...,...,...,...,...
288,constitucional,constitucionales,"Campo Moreno, Juan Carlos",Grupo Parlamentario Socialista,2020-02-19,-107.61,-107.10,ADJ,83,771
289,renovación,renovación,"Campo Moreno, Juan Carlos",Grupo Parlamentario Socialista,2020-02-19,-107.61,-107.10,NOUN,83,773
290,fortalecimiento,fortalecimiento,"Campo Moreno, Juan Carlos",Grupo Parlamentario Socialista,2020-02-19,-107.61,-107.10,NOUN,83,776
291,encontrar,encontrarán,"Campo Moreno, Juan Carlos",Grupo Parlamentario Socialista,2020-02-19,-107.61,-107.10,VERB,83,780


In [ ]:
jjjjj

In [2]:
palabras_lemm = 'palabras.pickle'
with open(palabras_lemm, "rb") as f:
    df_lemmas = pickle.load(f)

In [4]:
df_lemmas['lemma'] = df_lemmas['lemma'].str.lower()
df_lemmas = df_lemmas[df_lemmas['lemma'] != ' ']
df_lemmas.head(5)

,lemma,palabra,hablante,grupo parlamentario,fecha,szigriszt_pazos,flesch_reading_ease,tipo palabra,numero_discurso,orden
0,señor,Señor,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso,2020-02-19,-79.6,-79.01,PROPN,0,0
1,sánchez,Sánchez,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso,2020-02-19,-79.6,-79.01,PROPN,0,1
2,bloquear,bloquea,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso,2020-02-19,-79.6,-79.01,VERB,0,3
3,ayer,ayer,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso,2020-02-19,-79.6,-79.01,ADV,0,9
4,tener,tuvimos,"Casado Blanco, Pablo",Grupo Parlamentario Popular en el Congreso,2020-02-19,-79.6,-79.01,VERB,0,10


In [5]:
df_prueba = df_lemmas[['hablante', 'lemma']].head(100)
df_prueba.to_csv('pruebas.csv')

In [378]:
#Legibilidad
df_legibilidad = df_lemmas.groupby(['hablante', 'grupo parlamentario', 'numero_dicurso'])\
.agg({'szigriszt_pazos':max, 'flesch_reading_ease':max}).reset_index()
df_legibilidad
#

,hablante,grupo parlamentario,numero_dicurso,szigriszt_pazos,flesch_reading_ease
0,"Arrimadas García, Inés",Grupo Parlamentario Ciudadanos,4,-0.88,2.24
1,"Arrimadas García, Inés",Grupo Parlamentario Ciudadanos,6,11.70,14.47
2,"Bal Francés, Edmundo",Grupo Parlamentario Ciudadanos,80,29.66,33.15
3,"Bal Francés, Edmundo",Grupo Parlamentario Ciudadanos,82,40.29,43.89
4,"Bel Accensi, Ferran",Grupo Parlamentario Plural,69,-127.53,-127.30
...,...,...,...,...,...
79,"Sánchez Pérez-Castejón, Pedro",Grupo Parlamentario Socialista,8,-123.53,-122.87
80,"Ábalos Meco, José Luis",Grupo Parlamentario Socialista,40,-51.60,-50.33
81,"Ábalos Meco, José Luis",Grupo Parlamentario Socialista,42,-151.29,-151.19
82,"Álvarez de Toledo Peralta-Ramos, Cayetana",Grupo Parlamentario Popular en el Congreso,9,1.69,5.60


In [194]:
df_agrupado = df_lemmas.groupby(['lemma', 'hablante', 'grupo parlamentario', 'fecha', 'tipo palabra'])['palabra'].apply(list).reset_index()
df_agrupado['num_palabras'] = df_agrupado['palabra'].apply(len)
df_agrupado['palabras agrupadas'] = df_agrupado['palabra'].apply(lambda x: list(dict.fromkeys(x)))
df_agrupado.columns

Index(['lemma', 'hablante', 'grupo parlamentario', 'fecha', 'tipo palabra',
       'palabra', 'num_palabras', 'palabras agrupadas'],
      dtype='object')

In [196]:
df_sin_atipicos = df_agrupado[['hablante', 'lemma', 'num_palabras', 'fecha']].groupby(['lemma']).agg({'num_palabras':sum}).reset_index()
avg = df_sin_atipicos['num_palabras'].mean()
stDev = df_sin_atipicos['num_palabras'].std()
#descartamos todas las palabras que se hayan dicho menos de 5 veces
df_sin_atipicos = df_sin_atipicos[df_sin_atipicos['num_palabras'] >= 3]
limite_maximo = round(avg + (stDev * 1.5))
df_sin_atipicos = df_sin_atipicos[df_sin_atipicos['num_palabras'] < limite_maximo]
df_sin_atipicos = df_sin_atipicos.drop(columns=['num_palabras'])
df_sin_atipicos.columns
#

Index(['lemma'], dtype='object')

In [201]:
df_resultado = pd.merge(df_agrupado, df_sin_atipicos, on='lemma')
df_resultado = df_resultado[df_resultado['lemma'].apply(lambda x: not x[0].isalpha())]
df_resultado

,lemma,hablante,grupo parlamentario,fecha,tipo palabra,palabra,num_palabras,palabras agrupadas
1112268,°,"Calvo Poyato, Carmen",Grupo Parlamentario Socialista,2020-12-16,NUM,[°],1,[°]
1112269,°,"García Egea, Teodoro",Grupo Parlamentario Popular en el Congreso,2020-12-16,NUM,[°],1,[°]
1112270,°,"Márquez Guerrero, María",Grupo Parlamentario Confederal de Unidas Podem...,2020-12-15,NUM,[°],1,[°]
1112271,°,"Ribera Rodríguez, Teresa",Grupo Parlamentario Socialista,2021-05-13,NUM,[°],1,[°]
1112272,´,"De Meer Méndez, Rocío",Grupo Parlamentario VOX,2020-10-27,PROPN,[´],1,[´]
1112273,´,"Ribera Rodríguez, Teresa",Grupo Parlamentario Socialista,2020-05-13,PROPN,[´],1,[´]
1112274,´,"Ribera Rodríguez, Teresa",Grupo Parlamentario Socialista,2022-11-02,PROPN,[´],1,[´]
1112275,´,"Vidal Sáez, Aina",Grupo Parlamentario Confederal de Unidas Podem...,2021-03-25,PROPN,[´],1,[´]


In [ ]:
°´

In [165]:
len(df_agrupado) #antes: 2526813

2524872

In [166]:
df_lemmas_unicos = df_agrupado.groupby(['lemma']).agg({'palabras agrupadas':sum, 'num_palabras':sum, 'tipo palabra':list}).reset_index()
df_lemmas_unicos['palabras agrupadas unicas'] = df_lemmas_unicos['palabras agrupadas'].apply(lambda x: list(dict.fromkeys(x)))
df_lemmas_unicos['tipo palabra agg'] = df_lemmas_unicos['tipo palabra'].apply(lambda x: list(dict.fromkeys(x)))
df_lemmas_unicos = df_lemmas_unicos.drop('palabras agrupadas', axis=1)
df_lemmas_unicos['polaridad'] = df_lemmas_unicos.apply(devuelve_polaridad, axis=1)
df_lemmas_unicos['polaridad_redondeada'] = df_lemmas_unicos['polaridad'].apply(lambda x: round(x, 1))

In [167]:
len(df_lemmas_unicos) #71205

64683

In [ ]:
def inserta_hablante(row):
    # Conexion a la base de datos Neo4j
    graph = row['graph']

    # Insertar nodo
    return graph.run("CREATE (h:Hablante {nombre: $name, grupo_parlamentario: $grupo})",
                     name=row['hablante'], grupo=row['grupo parlamentario']).stats()['nodes_created']


def inserta_lemma(row):
    # Conexion a la base de datos Neo4j
    graph = row['graph']

    # Insertar nodo
    return graph.run(
        "CREATE (p:Palabra {lemma: $lemma, palabras_agrupadas: $grupo, tipos_palabra: $tipos, \
        polaridad: $pol, polaridad_round: $pol_round})",
        lemma=row['lemma'],
        grupo=row['palabras agrupadas unicas'],
        tipos=row['tipo palabra agg'],
        pol=row['polaridad'],
        pol_round=row['polaridad_redondeada']
        ).stats()['nodes_created']


In [141]:

df_resultados_lemmas = df_lemmas_unicos[['lemma', 'num_palabras', 'palabras agrupadas unicas', 
                                         'tipo palabra agg', 'polaridad', 'polaridad_redondeada']]
df_resultados_lemmas['graph'] = graph

results = []
for i in range(0, len(df_resultados_lemmas), 100):
    block = df_resultados_lemmas[i:i+100]
    results.extend(block.apply(inserta_lemma, axis=1).tolist())
df_lemmas_unicos['resultado'] = results


df_resultados_lemmas
#

,lemma,num_palabras,palabras agrupadas unicas,tipo palabra agg,polaridad,polaridad_redondeada,graph
12921,Olivia,11,[Olivia],[PROPN],0.993852,1.0,Graph('bolt://localhost:7687')
57562,prado,2,"[prados, Prados]",[ADJ],0.986813,1.0,Graph('bolt://localhost:7687')
14184,Prado,33,[Prado],[PROPN],0.986813,1.0,Graph('bolt://localhost:7687')
1711,Atocha,12,[Atocha],[PROPN],0.985217,1.0,Graph('bolt://localhost:7687')
15454,Retiro,4,[Retiro],[PROPN],0.979736,1.0,Graph('bolt://localhost:7687')
...,...,...,...,...,...,...,...
18328,Vergüenza,28,[Vergüenza],[PROPN],0.004377,0.0,Graph('bolt://localhost:7687')
59549,pésimo,97,"[pésima, pésimo, pésimas]",[ADJ],0.003942,0.0,Graph('bolt://localhost:7687')
23981,asqueroso,7,"[asquerosas, asquerosos, asqueroso, asquerosa]","[ADJ, NOUN]",0.002213,0.0,Graph('bolt://localhost:7687')
41974,fatal,75,"[fatales, fatal]","[ADJ, ADV]",0.001200,0.0,Graph('bolt://localhost:7687')


In [111]:
df_hablantes_unicos = df_agrupado.drop_duplicates(['hablante', 'grupo parlamentario'])
df_resultados_hablantes = df_hablantes_unicos[['hablante', 'grupo parlamentario']]
df_resultados_hablantes['graph'] = graph
df_resultados_hablantes['resultado'] = df_resultados_hablantes.apply(inserta_hablante, axis=1)
df_resultados_hablantes

,hablante,grupo parlamentario,resultado
0,"Alonso-Cuevillas i Sayrol, Jaume",Grupo Parlamentario Plural,1
1,"Bel Accensi, Ferran",Grupo Parlamentario Plural,1
2,"Boadella Esteve, Genís",Grupo Parlamentario Plural,1
3,"Callejas Cano, Juan Antonio",Grupo Parlamentario Popular en el Congreso,1
4,"Casares Hontañón, Pedro",Grupo Parlamentario Socialista,1
...,...,...,...
212062,"González Laso, Natividad",Grupo Parlamentario Socialista,1
1453284,"González Muñoz, Ángel Luis",Grupo Parlamentario Popular en el Congreso,1
1453289,"Quintanilla Navarro, Miguel Ángel",Grupo Parlamentario Popular en el Congreso,1
1453290,"Romaní Cantera, José Ignacio",Grupo Parlamentario Popular en el Congreso,1


In [153]:
def inserta_relacion_dice(row):
    # Conexion a la base de datos Neo4j
    graph = row['graph']

    # Insertar nodo
    return graph.run(
        'MATCH (n:Hablante) \
         MATCH (p:Palabra) \
         WHERE n.nombre =  $nombre \
         AND p.lemma = $lemma \
         MERGE (n)-[r:DICE {fecha: $fecha, veces: $veces}]->(p) \
         RETURN COUNT(r)',
        nombre = row['hablante'],
        lemma=row['lemma'],
        fecha=row['fecha'],
        veces=row['num_palabras']
        )

In [185]:
df_atipicos = df_agrupado[['hablante', 'lemma', 'num_palabras', 'fecha']].groupby(['lemma']).agg({'num_palabras':sum}).reset_index()
print(len(df_atipicos))
avg = df_atipicos['num_palabras'].mean()
stDev = df_atipicos['num_palabras'].std()
#descartamos todas las palabras que se hayan dicho menos de 5 veces
df_atipicos = df_atipicos[df_atipicos['num_palabras'] >= 3]
limite_maximo = round(avg + (stDev * 1.5))
df_atipicos = df_atipicos[df_atipicos['num_palabras'] < limite_maximo]
len(df_atipicos)
#

64683


29488

In [177]:
stDev

525.1768376935394

In [154]:
df_relaciones = df_agrupado[['hablante', 'lemma', 'num_palabras', 'fecha']]
df_relaciones['graph'] = graph
results = []
for i in range(0, len(df_relaciones), 100):
    block = df_relaciones[i:i+100]
    results.extend(block.apply(inserta_relacion_dice, axis=1).tolist())
df_relaciones['resultado'] = results


KeyboardInterrupt: 

In [ ]:
df_relaciones

In [ ]:
df_lemmas.sort_values(by=['hablante', 'grupo parlamentario', 'fecha', 'numero_dicurso', 'orden', 'lemma'], inplace=True, ascending=True)
df_ordenado = df_lemmas.groupby(['hablante', 'grupo parlamentario', 'fecha', 'numero_dicurso'], group_keys=True)
df_ordenado = df_ordenado.apply(lambda x: x.assign(lemma_siguiente=x['lemma'].shift(-1)))
df_ordenado[['lemma', 'lemma_siguiente', 'hablante', 'grupo parlamentario', 'fecha', 'numero_discurso', 'orden']]
#

In [70]:
len(df_lemmas_unicos)

3429

In [ ]:
import pandas as pd

# Crea un DataFrame de ejemplo
df = pd.DataFrame({'A': [1, 2, 3, 4],
                   'B': [10, 20, 30, 40],
                   'C': [100, 200, 300, 400]})

# Define la función a aplicar a cada fila
def my_func(row):
    return row['A'] + row['B']

# Aplica la función a cada fila del DataFrame y guarda el resultado en una nueva columna
df['D'] = df.apply(my_func, axis=1)

# Muestra el nuevo DataFrame resultante
print(df)

In [42]:
import dask.dataframe as dd

# Crea un DataFrame de ejemplo
df = pd.DataFrame({'A': [1, 2, 3, 4],
                   'B': [10, 20, 30, 40],
                   'C': [100, 200, 300, 400]})

# Convierte el DataFrame a un DataFrame de Dask
dask_df = dd.from_pandas(df, npartitions=4)

# Define la función a aplicar a cada fila
def my_func(row):
    return row['A'] + row['B']

# Aplica la función al DataFrame de Dask
dask_df_result = dask_df.map_partitions(lambda df: df.apply(my_func, axis=1), meta=('0', 'f8'))

# Convierte el DataFrame de Dask a un DataFrame de Pandas
df_result = dask_df_result.compute()

# Muestra el nuevo DataFrame resultante
print(df_result)



0    11
1    22
2    33
3    44
Name: 0, dtype: int64


In [11]:
%%time
from sentiment_analysis_spanish import sentiment_analysis
sentiment = sentiment_analysis.SentimentAnalysisSpanish()
print(sentiment.sentiment("España"))

0.7627843246566761
CPU times: total: 5.52 s
Wall time: 5.58 s


In [81]:
global sentiment
sentiment = sentiment_analysis.SentimentAnalysisSpanish()

In [82]:
def devuelve_polaridad(row):
    return sentiment.sentiment(row['lemma'])

In [44]:
len(df_lemmas_unicos)

3274

In [83]:
df_lemmas_unicos_20 = df_lemmas_unicos[:4]

In [85]:
df_lemmas_unicos_20['polaridad'] = df_lemmas_unicos_20.apply(devuelve_polaridad, axis=1)
df_lemmas_unicos_20

,lemma,num_palabras,tipo palabra,palabras agrupadas unicas,tipo palabra agg,polaridad
197,Gobierno,194,"[PROPN, PROPN, PROPN, PROPN, PROPN, PROPN, PRO...",[Gobierno],[PROPN],0.497892
156,España,103,"[PROPN, PROPN, PROPN, PROPN, PROPN, PROPN, PRO...",[España],[PROPN],0.762784
2943,señor,97,"[NOUN, NOUN, NOUN, NOUN, VERB, NOUN, INTJ, NOU...","[señor, señores, Señor]","[NOUN, VERB, INTJ, ADJ]",0.130024
735,año,93,"[NOUN, NOUN, NOUN, NOUN, NOUN, NOUN, NOUN, NOU...","[años, año]",[NOUN],0.412465


In [78]:
from sentiment_analysis_spanish import sentiment_analysis
sentiment = sentiment_analysis.SentimentAnalysisSpanish()
print(sentiment.sentiment("me gusta la tombola es genial"))

0.9304396176531412


In [79]:
%%time
sentiment.sentiment("me gusta la tombola es genial")

CPU times: total: 0 ns
Wall time: 0 ns


0.9304396176531412